#  Loading the dataset and creating Nlist 

In [4]:
import pandas as pd
import re
import os
import pyodbc
from sqlalchemy import create_engine

# Define the server, database, username, and password
server = '192.168.3.5'
database = 'DB_Glomis_02may'
username = 'glomistest1'
password = 'glomistest1'

# Define the connection string
conn_str = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=SQL Server'
engine = create_engine(conn_str)

# Read Shibboleth(IDP) URLs and create a set for filtering
IDP_df = pd.read_csv('ACCOUNTS FEED.20246.csv')
idp_urls = set(IDP_df['Shibboleth(IDP)'])

# Define the target folder containing log files
target_folder = 'D:/Ijour_logFile/June2024'

# Function to extract date from filename
def extract_date_from_filename(filename):
    date_pattern = r'u_ex(\d{2})(\d{2})(\d{2})\.log'
    match = re.match(date_pattern, filename)
    if match:
        day = match.group(3)
        month = match.group(2)
        year = match.group(1)
        return f'{day}/{month}/20{year}'
    return None

# Function to extract relevant part of the URL
def extract_relevant_part(url):
    try:
        parts = url.split('/')
        if len(parts) > 2:
            domain_parts = parts[2].split('.')
            return '.'.join(domain_parts[-2:])  # Extracting the last two parts of the domain
        else:
            return None
    except Exception as e:
        print(f"Error processing URL: {url}, Error: {e}")
        return None

# Process each log file in the target folder
for log_file in os.listdir(target_folder):
    log_path = os.path.join(target_folder, log_file)
    if os.path.isfile(log_path):
        date_string = extract_date_from_filename(log_file)
        
        if date_string:
            # Query the database to check if this date has already been processed
            query = f"SELECT COUNT(*) FROM A_Downloads_First_Second_Process1 WHERE [Date] = '{date_string}'"
            result = pd.read_sql(query, engine)
            
            if result.iloc[0, 0] == 0:  # If the date is not found in the database
                data = []
                
                try:
                    with open(log_path, 'r', encoding='utf-8') as file:
                        header_found = False
                        for line in file:
                            if line.startswith('#Fields:'):
                                header_found = True
                                columns = line.strip().split(' ')[1:]
                            elif header_found and not line.startswith('#'):
                                data.append(line.strip().split(' '))
                except UnicodeDecodeError:
                    with open(log_path, 'r', encoding='latin-1') as file:
                        header_found = False
                        for line in file:
                            if line.startswith('#Fields:'):
                                header_found = True
                                columns = line.strip().split(' ')[1:]
                            elif header_found and not line.startswith('#'):
                                data.append(line.strip().split(' '))
                
                df1 = pd.DataFrame(data, columns=columns)
                
                if 'date' in df1.columns:
                    df1.drop(columns=['date'], inplace=True)
                
                df1.insert(0, 'date', date_string)
                df1['sc-status'] = df1['sc-status'].astype('int64')
                
                # Example: Filtering based on Shibboleth(IDP) URLs
                # filtered_df = df1[df1['cs(Referer)'].apply(extract_relevant_part).isin(idp_urls)]
                # Further processing on filtered_df if needed

                # Proceed with any other processing steps required
                # ...
engine.dispose()
df1

,date,time,s-sitename,s-computername,s-ip,cs-method,cs-uri-stem,cs-uri-query,s-port,cs-username,...,cs(User-Agent),cs(Cookie),cs(Referer),cs-host,sc-status,sc-substatus,sc-win32-status,sc-bytes,cs-bytes,time-taken
0,03/06/2024,18:30:00,W3SVC7,WIN-SGLO1ATQVG5,192.191.146.10,GET,/ijor.aspx,target=ijor:ijmss&volume=3&issue=5&article=012...,443,-,...,"Mozilla/5.0+AppleWebKit/537.36+(KHTML,+like+Ge...",-,-,www.indianjournals.com,302,0,0,195148,352,1691
1,03/06/2024,18:30:00,W3SVC7,WIN-SGLO1ATQVG5,192.191.146.10,GET,/ijor.aspx,target=ijor:ijmss&volume=3&issue=5&article=012,443,-,...,"Mozilla/5.0+AppleWebKit/537.36+(KHTML,+like+Ge...",ASP.NET_SessionId=s2erx3pqokc0lzyb2kwx0men;+pa...,-,www.indianjournals.com,200,0,0,56436,427,519
2,03/06/2024,18:30:00,W3SVC7,WIN-SGLO1ATQVG5,192.191.146.10,GET,/redirectToAD.aspx,id=MgAzADcANQA=&adAddress=http://www.indianjou...,443,-,...,Mozilla/5.0+(compatible;+MJ12bot/v1.4.8;+http:...,-,-,www.indianjournals.com,302,0,0,677,486,237
3,03/06/2024,18:30:01,W3SVC7,WIN-SGLO1ATQVG5,192.191.146.10,GET,/ijor.aspx,target=ijor:rjpt&volume=8&issue=11&article=021,443,-,...,Mozilla/5.0+(compatible;+AhrefsBot/7.0;++http:...,-,-,indianjournals.com,200,0,0,54116,229,862
4,03/06/2024,18:30:01,W3SVC7,WIN-SGLO1ATQVG5,192.191.146.10,GET,/Mobile/home.aspx,-,443,-,...,Mozilla/5.0+(Linux;+Android+12;+Pixel+4)+Apple...,-,https://www.indianjournals.com/ijor.aspx?artic...,www.indianjournals.com,500,0,0,4949,524,1987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514740,03/06/2024,18:29:57,W3SVC7,WIN-SGLO1ATQVG5,192.191.146.10,GET,/ijor.aspx,target=ijor:ijcmiid&volume=42&issue=2spl&artic...,443,-,...,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+Appl...,-,-,www.indianjournals.com,200,0,0,220662,357,2390
514741,03/06/2024,18:29:58,W3SVC7,WIN-SGLO1ATQVG5,192.191.146.10,GET,/ijor.aspx,target=ijor:ijcmiid&volume=25&issue=2&article=...,443,-,...,Mozilla/5.0+(compatible;+MJ12bot/v1.4.8;+http:...,-,-,www.indianjournals.com,302,0,64,65876,446,3390
514742,03/06/2024,18:29:58,W3SVC7,WIN-SGLO1ATQVG5,192.191.146.10,GET,/ijor.aspx,issue=10&target=ijor:sajmmr&type=toc&volume=8,443,-,...,Mozilla/5.0+(Macintosh;+Intel+Mac+OS+X+10_10_1...,-,-,www.indianjournals.com,200,0,0,51508,348,1984
514743,03/06/2024,18:29:58,W3SVC7,WIN-SGLO1ATQVG5,192.191.146.10,GET,/ijor.aspx,target=ijor:ajm&volume=8&issue=3&article=055,80,-,...,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+Appl...,-,-,www.indianjournals.com,301,0,0,499,349,234


# Creating the IP Mapping for Nlist  

In [6]:
# Function to extract keywords from Shibboleth URLs
def extract_keywords(url):
    try:
        parts = url.split('/')
        if len(parts) > 2:
            domain_parts = parts[2].split('.')
            if len(domain_parts) >= 2:
                return '.'.join(domain_parts[:2])  # Extract the first two parts of the domain
        return None
    except Exception as e:
        print(f"Error processing URL: {url}, Error: {e}")
        return None

# Read Shibboleth(IDP) URLs and create a set of keywords
IDP_df1 = pd.read_csv('ACCOUNTS FEED.20246.csv')
IDP_df = IDP_df1[IDP_df1['Shibboleth(IDP)'].notna() & (IDP_df1['Shibboleth(IDP)'].str.strip() != '')]
idp_keywords = set(IDP_df['Shibboleth(IDP)'].apply(extract_keywords))
# Add a new column 'Keyword_use' to IDP_df containing the corresponding keywords
IDP_df=IDP_df.copy()
IDP_df.loc[:, 'Keyword_use'] = IDP_df['Shibboleth(IDP)'].apply(extract_keywords)
# Assuming you have already loaded the 'filtered_df' DataFrame containing the relevant data

# Create a dictionary to map Shibboleth(IDP) keywords to client names
client_mapping = dict(zip(IDP_df['Keyword_use'], IDP_df['ClientName']))

# Function to extract client name from Referer URL based on Shibboleth(IDP) keyword mapping
def extract_client_name(url):
    for keyword, client_name in client_mapping.items():
        if keyword in url:
            return client_name
    return 'Other IDP'  # If no match found
filtered_df = df1.copy()
# Apply the function to create the 'Client Name' column
filtered_df['Client Name'] = filtered_df['cs(Referer)'].apply(extract_client_name)

# Filter the DataFrame to include only 'c-ip' and 'Client Name' columns
ip_mapping = filtered_df.loc[filtered_df['Client Name'] != 'Other IDP', ['c-ip', 'Client Name']]

# Optionally, remove duplicate IP addresses
ip_mapping = ip_mapping.drop_duplicates()

# Display the resulting DataFrame
ip_mapping

,c-ip,Client Name
141767,59.184.227.232,Inflibnet Centre
146600,203.115.126.34,Inflibnet Centre
148647,223.185.232.111,Inflibnet Centre
154775,220.227.158.121,Inflibnet Centre
155726,103.72.171.170,Inflibnet Centre
...,...,...
379933,117.249.218.220,Inflibnet Centre
393829,152.58.26.153,Inflibnet Centre
396932,49.37.96.190,Inflibnet Centre
401083,157.42.0.22,Inflibnet Centre


# Categorization ( BOTS AND QUERY ) 

In [165]:
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
import pickle
import re 
# Final UDF ( NEW )
def categorize_query(query, sc_status):
    if query.startswith('target=ijor') and 'volume' in query and 'issue' in query and 'article' in query and \
       (query.endswith('type=pdf') or query.endswith('type=fulltext')):
        if sc_status == 200:
            return 'Download'
        else:
            return 'Denial'
    elif 'volume' in query and 'issue' in query and 'article' in query:
        if sc_status == 200:
            return 'Successful Investigation'
        else:
            return 'Unsuccessful Investigation'
    else:
        return 'Other'
df_filtered = df1[['date','time','cs(Cookie)','cs(User-Agent)','c-ip','cs-uri-query','cs(Referer)','sc-status','time-taken']]

# Displaying the first few rows of the new DataFrame
# Keep only unique entries in df_filtered
unique_df_filtered_x = df_filtered.drop_duplicates(subset=['date','time','cs(Cookie)','cs(User-Agent)','c-ip', 'cs-uri-query','cs(Referer)', 'sc-status'])
# Load the compiled regex patterns from the Pickle file
with open('bot_patterns.pkl', 'rb') as file:
    regex_patterns = pickle.load(file)
# Define a function to check if a user agent string matches any of the bot patterns
def check_bot(user_agent):
    return is_bot(user_agent)
def is_bot(user_agent):
    return any(pattern.search(user_agent) for pattern in regex_patterns)

# Create a ThreadPoolExecutor
executor = ThreadPoolExecutor(max_workers=20)  # Adjust the number of workers as needed
unique_df_filtered=unique_df_filtered_x.copy()
# Submit the tasks to the executor
futures = [executor.submit(check_bot, user_agent) for user_agent in unique_df_filtered['cs(User-Agent)']]

# Retrieve the results as they become available
bot_results = [future.result() for future in futures]

# Assign the results to the DataFrame
unique_df_filtered['is_bot'] = bot_results

# Display the DataFrame
unique_df_filtered
df_filtered1 = unique_df_filtered.copy()

df_filtered1['Query_Category'] = df_filtered1.apply(lambda row: categorize_query(row['cs-uri-query'], row['sc-status']), axis=1)
df_filtered1

,date,time,cs(Cookie),cs(User-Agent),c-ip,cs-uri-query,cs(Referer),sc-status,time-taken,is_bot,Query_Category
0,28/04/2024,18:30:00,-,"Mozilla/5.0+AppleWebKit/537.36+(KHTML,+like+Gecko;+compatible;+bingbot/2.0;++http://www.bing.com/bingbot.htm)+Chrome/116.0.1938.76+Safari/537.36",40.77.167.136,target=ijor:ijes&volume=3&issue=4&article=010&type=pdf,-,503,812,True,Denial
1,28/04/2024,18:30:00,-,"Mozilla/5.0+(Macintosh;+Intel+Mac+OS+X+10_15_7)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",128.240.225.12,target=ijor:ajmr&volume=5&issue=4&article=002,https://scholar.google.com/,503,531,False,Unsuccessful Investigation
2,28/04/2024,18:30:00,_ga=GA1.1.2142528575.1698317661;+_ga_RDCC5F3593=GS1.1.1710683710.6.1.1710683744.0.0.0;+_ga_G5TCNFJCYP=GS1.1.1710683744.1.0.1710683744.0.0.0,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36+Edg/124.0.0.0",223.185.49.160,target=ijor:apps&volume=11&issue=2&article=024,https://scholar.google.com/,503,218,False,Unsuccessful Investigation
3,28/04/2024,18:30:00,-,"Mozilla/5.0+(Linux;+U;+Android+11;+AE9010+Build/RP1A.201005.001;+wv)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Version/4.0+Chrome/124.0.6367.54+Mobile+Safari/537.36+OPR/80.0.2254.71401",42.0.4.192,-,https://www.indianjournals.com/ijor.aspx?target=ijor:jiafm&volume=32&issue=1&article=007,503,593,False,Other
4,28/04/2024,18:30:00,-,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",152.58.4.115,-,https://www.indianjournals.com/ijor.aspx?target=ijor:jgc&volume=13&issue=2&article=003,503,374,False,Other
...,...,...,...,...,...,...,...,...,...,...,...
606843,28/04/2024,18:29:54,-,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",109.225.176.40,target=ijor:rjst&volume=14&issue=2&article=007,-,503,421,False,Unsuccessful Investigation
606844,28/04/2024,18:29:55,-,facebookexternalhit/1.1+(+http://www.facebook.com/externalhit_uatext.php),69.63.184.4,target=ijor:ijan&volume=30&issue=4&article=003,-,503,249,True,Unsuccessful Investigation
606845,28/04/2024,18:29:57,-,Mozilla/5.0+(X11;+Linux+x86_64;+rv:103.0)+Gecko/20100101+Firefox/103.0,146.70.172.71,target=ijor:tin&volume=10&issue=3&article=003,-,503,687,False,Unsuccessful Investigation
606846,28/04/2024,18:29:57,-,Mozilla/5.0+(compatible;+BLEXBot/1.0;++http://webmeup-crawler.com/),162.55.85.224,target=ijor:rjpp&volume=5&issue=6&article=005,-,503,250,True,Unsuccessful Investigation


# Calculation of bots

In [166]:
df_total_bot = unique_df_filtered[unique_df_filtered['is_bot'] == True]
df_total_bot 


,date,time,cs(Cookie),cs(User-Agent),c-ip,cs-uri-query,cs(Referer),sc-status,time-taken,is_bot
0,28/04/2024,18:30:00,-,"Mozilla/5.0+AppleWebKit/537.36+(KHTML,+like+Gecko;+compatible;+bingbot/2.0;++http://www.bing.com/bingbot.htm)+Chrome/116.0.1938.76+Safari/537.36",40.77.167.136,target=ijor:ijes&volume=3&issue=4&article=010&type=pdf,-,503,812,True
6,28/04/2024,18:30:00,-,"Mozilla/5.0+(iPhone;+CPU+iPhone+OS+7_0+like+Mac+OS+X)+AppleWebKit/537.51.1+(KHTML,+like+Gecko)+Version/7.0+Mobile/11A465+Safari/9537.53+(compatible;+bingbot/2.0;++http://www.bing.com/bingbot.htm)",40.77.167.136,target=ijor:rjpt&volume=14&issue=9&article=038,-,503,562,True
10,28/04/2024,18:30:00,-,facebookexternalhit/1.1+(+http://www.facebook.com/externalhit_uatext.php),69.63.184.17,target=ijor:gjfsm&volume=4&issue=4&article=001,-,503,625,True
12,28/04/2024,18:30:02,-,Mozilla/5.0+(compatible;+Baiduspider/2.0;++http://www.baidu.com/search/spider.html),220.181.108.111,target=ijor:ahhs&type=current_issue,-,503,1187,True
13,28/04/2024,18:30:03,-,Mozilla/5.0+(compatible;+DataForSeoBot/1.0;++https://dataforseo.com/dataforseo-bot),136.243.228.177,target=ijor:ahhs&volume=8&issue=1and2&article=001,-,503,818,True
...,...,...,...,...,...,...,...,...,...,...
606836,28/04/2024,18:29:51,-,"Mozilla/5.0+AppleWebKit/537.36+(KHTML,+like+Gecko;+compatible;+bingbot/2.0;++http://www.bing.com/bingbot.htm)+Chrome/116.0.1938.76+Safari/537.36",157.55.39.200,target=ijor:ar&volume=32&issue=2&article=009&type=subscribearticle,-,503,328,True
606837,28/04/2024,18:29:51,-,"Mozilla/5.0+AppleWebKit/537.36+(KHTML,+like+Gecko;+compatible;+GPTBot/1.0;++https://openai.com/gptbot)",52.233.106.96,target=ijor:ijass&volume=4&issue=1&article=006&type=pdf,-,503,296,True
606842,28/04/2024,18:29:52,-,facebookexternalhit/1.1+(+http://www.facebook.com/externalhit_uatext.php),173.252.83.25,target=ijor:jes&type=eboard,-,503,359,True
606844,28/04/2024,18:29:55,-,facebookexternalhit/1.1+(+http://www.facebook.com/externalhit_uatext.php),69.63.184.4,target=ijor:ijan&volume=30&issue=4&article=003,-,503,249,True


# Define check_scripted and categorize_client(ISP) as UDF , using check_scripted on slices of dataframes after making copies 

In [167]:
# Define the UDF to check for the specified substrings in the query
def check_scripted(query):
    if '%' in query or '&amp' in query or 'ccode' in query:
        return 'scripted'
    else:
        return ''

# List of ISP identifiers
isp_identifiers = [
    'limited', 'network', 'llc','ltd', 'telecom','telephone', 
    'company', 'service', 'telecommunications', 'communications corporation',
    'host', 'cable datacom limited', 'foundation', 'technologies', 'retail',
    'services', 'co. ltd.', 'dotcom', 'broadband', 'network internet',
    'corp.', 'inc.','inc','s.a.s.', 'plc', 'telenet', 'cellular', 'vodafone', 
    'telekom', 'wireless', 'pvt.','ltd.', 'digital', 'verizon', 'cables', 
    'skybroadband', 'communications','corporation','OVH SAS'
]

# Specific companies to flag
specific_companies = {
    'google': 'Google',
    'microsoft': 'Microsoft',
    'amazon': 'Amazon'
}


# Function to check the category of the client name
def categorize_client(client_name):
    client_name_lower = client_name.lower()
    for company, flag in specific_companies.items():
        if company in client_name_lower:
            return flag
    for isp in isp_identifiers:
        if isp in client_name_lower:
            return 'ISP'
    return 'Other'
df = df_filtered1
# Filter queries based on their categories
download_queries = df[df['Query_Category'] == 'Download']
successful_investigation_queries = df[df['Query_Category'] == 'Successful Investigation']
unsuccessful_investigation_queries = df[df['Query_Category'] == 'Unsuccessful Investigation']
other_queries = df[df['Query_Category'] == 'Other']
denial = df[df['Query_Category'] == 'Denial']

download_queries = download_queries.copy()
successful_investigation_queries = successful_investigation_queries.copy()
unsuccessful_investigation_queries = unsuccessful_investigation_queries.copy()
other_queries = other_queries.copy()
denial = denial.copy()

download_queries.loc[:,'scripted'] = download_queries['cs-uri-query'].apply(check_scripted)
successful_investigation_queries.loc[:,'scripted'] = successful_investigation_queries['cs-uri-query'].apply(check_scripted)
unsuccessful_investigation_queries.loc[:,'scripted'] = unsuccessful_investigation_queries['cs-uri-query'].apply(check_scripted)
other_queries.loc[:,'scripted'] = other_queries['cs-uri-query'].apply(check_scripted)
denial.loc[:,'scripted'] = denial['cs-uri-query'].apply(check_scripted)

# Display the number of queries in each category
print("Number of Download Queries:", len(download_queries))
print("Number of Successful Investigation Queries:", len(successful_investigation_queries))
print("Number of Unsuccessful Investigation Queries:", len(unsuccessful_investigation_queries))
print("Number of Other Queries:", len(other_queries))

Number of Download Queries: 2679
Number of Successful Investigation Queries: 119258
Number of Unsuccessful Investigation Queries: 200286
Number of Other Queries: 227697


# Complete Analysis of Downloads 

In [168]:
######################################################################################################
######################################################################################################
######################################################################################################

#           Downloads 

######################################################################################################
######################################################################################################
######################################################################################################
import pandas as pd
import ipaddress
from datetime import datetime, timedelta 
# Group by the specified columns and select unique 'time' values within each group
df_downloads = download_queries.groupby(['date','cs-uri-query' ,'cs(User-Agent)','c-ip','cs(Cookie)','cs(Referer)','sc-status','is_bot','scripted'])['time'].unique().reset_index()

df_downloads['time'] = df_downloads['time'].apply(lambda x: [datetime.strptime(t, '%H:%M:%S') for t in x])

# Define the time window for double clicks (30 seconds)
time_window = timedelta(seconds=30)

# Initialize a list to store the indices of clicks to be removed
indices_to_remove = []

# Iterate through each row in the DataFrame
for index, row in df_downloads.iterrows():
    # Get the list of time values for the current row
    times = row['time']
    
    # Iterate through consecutive pairs of time values
    for i in range(len(times) - 1):
        # Calculate the time difference between consecutive clicks
        time_diff = times[i + 1] - times[i]
        
        # Check if the time difference is within the time window
        if time_diff <= time_window:
            # Mark the later click for removal
            indices_to_remove.append(index)

# Drop the rows marked for removal
df_filtered = df_downloads.drop(indices_to_remove)

# Keep only the first entry for each set of consecutive clicks within the time window
df_filtered['time'] = df_filtered['time'].apply(lambda x: x[0])

# Reset the index of the filtered DataFrame
df_filtered.reset_index(drop=True, inplace=True)

# Assuming df_filtered is your DataFrame
df_downloads = df_filtered.copy()

# Extract the time part from datetime objects
df_downloads['time'] = df_downloads['time'].dt.strftime('%H:%M:%S')
df_downloads['Query_Category'] = 'Download'
# Display the DataFrame with the time values extracted
# df_downloads ( view this )

def ip_to_decimal(ip):
    return int(ipaddress.IPv4Address(ip))

# Apply the function to the 'c-ip' column and create a new column 'c-ip Decimal'
df_downloads['c-ip Decimal'] = df_downloads['c-ip'].apply(ip_to_decimal)
df_downloads

,date,cs-uri-query,cs(User-Agent),c-ip,cs(Cookie),cs(Referer),sc-status,is_bot,scripted,time,Query_Category,c-ip Decimal
0,28/04/2024,target=ijor%3Agjfsm&volume=2&issue=2&article=006&type=fulltext,"Mozilla/5.0+AppleWebKit/537.36+(KHTML,+like+Gecko;+compatible;+bingbot/2.0;++http://www.bing.com/bingbot.htm)+Chrome/116.0.1938.76+Safari/537.36",52.167.144.19,-,-,200,True,scripted,22:59:10,Download,883396627
1,28/04/2024,target=ijor%3Aijan&volume=12&issue=2&article=005&type=pdf,facebookexternalhit/1.1+(+http://www.facebook.com/externalhit_uatext.php),173.252.83.28,-,-,200,True,scripted,01:23:50,Download,2918994716
2,28/04/2024,target=ijor%3Aijh&volume=31&issue=1&article=024&type=pdf,facebookexternalhit/1.1+(+http://www.facebook.com/externalhit_uatext.php),173.252.79.5,-,-,200,True,scripted,18:40:18,Download,2918993669
3,28/04/2024,target=ijor%3Aijh&volume=31&issue=1&article=024&type=pdf,facebookexternalhit/1.1+(+http://www.facebook.com/externalhit_uatext.php),173.252.83.10,-,-,200,True,scripted,18:40:14,Download,2918994698
4,28/04/2024,target=ijor%3Aijh&volume=31&issue=1&article=024&type=pdf,facebookexternalhit/1.1+(+http://www.facebook.com/externalhit_uatext.php),173.252.95.12,-,-,200,True,scripted,18:40:36,Download,2918997772
...,...,...,...,...,...,...,...,...,...,...,...,...
2457,28/04/2024,target=ijor:zijbemr&volume=4&issue=6&article=018&type=pdf,Googlebot/2.1+(+http://www.google.com/bot.html),66.249.66.160,-,-,200,True,,07:53:58,Download,1123631776
2458,28/04/2024,target=ijor:zijmr&volume=2&issue=12&article=019&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",152.58.146.84,-,https://iask.ai/,200,False,,15:26:05,Download,2553975380
2459,28/04/2024,target=ijor:zijmr&volume=2&issue=5&article=017&type=pdf,"Mozilla/5.0+AppleWebKit/537.36+(KHTML,+like+Gecko;+compatible;+bingbot/2.0;++http://www.bing.com/bingbot.htm)+Chrome/116.0.1938.76+Safari/537.36",207.46.13.153,-,-,200,True,,23:28:08,Download,3475901849
2460,28/04/2024,target=ijor:zijmr&volume=2&issue=5&article=022&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",49.206.129.148,ASP.NET_SessionId=i0btfg12oiwaeo3rktcpdogw;+_gid=GA1.2.1512945033.1714279193;+_gat_gtag_UA_104381405_2=1;+_ga_RDCC5F3593=GS1.1.1714279192.1.0.1714279192.0.0.0;+_ga=GA1.1.1413538317.1714279193,https://www.indianjournals.com/ijor.aspx?target=ijor:zijmr&volume=2&issue=5&article=022,200,False,,04:39:56,Download,835617172


# Processes  :  IP based Identification 

In [169]:

import time
import pandas as pd 
df_clients=pd.read_excel('clients excel.xlsx')
df_clients
df_downloads['client name'] = ''

# Record the start time
start_time = time.time()

# Convert 'Start Decimal' and 'End Decimal' columns to numpy arrays for faster computation
start_decimals_df_clients = df_clients['Start Decimal'].values
end_decimals_df_clients = df_clients['End Decimal'].values
client_names_df_clients = df_clients['client name'].values

# Initialize the 'process' column to None
df_downloads['process'] = ''

# Iterate over each row in df5 to find the corresponding client name
for index, row in df_downloads.iterrows():
    c_ip_decimal = row['c-ip Decimal']
    current_client_name = row['client name']
    
    # Check if the client name is blank
    if pd.isnull(current_client_name) or current_client_name.strip() == '':
        # Use numpy array operations to check if c-ip Decimal falls within the range of start and end decimals in df8
        mask = (start_decimals_df_clients <= c_ip_decimal) & (c_ip_decimal <= end_decimals_df_clients)
        
        # If there's a match, assign the corresponding client name to df5 and mark the process
        if mask.any():
            df_downloads.at[index, 'client name'] = client_names_df_clients[mask][0]
            df_downloads.at[index, 'process'] = 'First Process'
            
# Record the end time
end_time1 = time.time()

# Calculate the total execution time
execution_time1 = end_time1 - start_time

print("Total execution time:", execution_time1, "seconds")

# Display the updated df5 with the Client Name and Process columns
ip_mapping1.rename(columns={'Client Name': 'client name'}, inplace=True)
# Assuming df9 is your IP mapping DataFrame and df_downloads is your main DataFrame

# Create a dictionary mapping IP addresses to client names
ip_client_mapping = dict(zip(ip_mapping1['c-ip'], ip_mapping1['client name']))

# Iterate over each row in df_downloads to map IP addresses to client names and update the process column
for index, row in df_downloads.iterrows():
    c_ip = row['c-ip']
    current_client_name = row['client name']
    current_process = row['process']
    
    # Check if the client name is empty or NaN
    if pd.isnull(current_client_name) or current_client_name.strip() == '':
        # Check if the IP address exists in the ip_client_mapping dictionary
        if c_ip in ip_client_mapping:
            # Assign the client name
            df_downloads.at[index, 'client name'] = ip_client_mapping[c_ip]
            # Update the process column to 'process two'
            df_downloads.at[index, 'process'] = 'Second Process'

# Display the DataFrame with the updated columns
df_downloads

Total execution time: 0.11035394668579102 seconds


,date,cs-uri-query,cs(User-Agent),c-ip,cs(Cookie),cs(Referer),sc-status,is_bot,scripted,time,Query_Category,c-ip Decimal,client name,process
0,28/04/2024,target=ijor%3Agjfsm&volume=2&issue=2&article=006&type=fulltext,"Mozilla/5.0+AppleWebKit/537.36+(KHTML,+like+Gecko;+compatible;+bingbot/2.0;++http://www.bing.com/bingbot.htm)+Chrome/116.0.1938.76+Safari/537.36",52.167.144.19,-,-,200,True,scripted,22:59:10,Download,883396627,,
1,28/04/2024,target=ijor%3Aijan&volume=12&issue=2&article=005&type=pdf,facebookexternalhit/1.1+(+http://www.facebook.com/externalhit_uatext.php),173.252.83.28,-,-,200,True,scripted,01:23:50,Download,2918994716,,
2,28/04/2024,target=ijor%3Aijh&volume=31&issue=1&article=024&type=pdf,facebookexternalhit/1.1+(+http://www.facebook.com/externalhit_uatext.php),173.252.79.5,-,-,200,True,scripted,18:40:18,Download,2918993669,,
3,28/04/2024,target=ijor%3Aijh&volume=31&issue=1&article=024&type=pdf,facebookexternalhit/1.1+(+http://www.facebook.com/externalhit_uatext.php),173.252.83.10,-,-,200,True,scripted,18:40:14,Download,2918994698,,
4,28/04/2024,target=ijor%3Aijh&volume=31&issue=1&article=024&type=pdf,facebookexternalhit/1.1+(+http://www.facebook.com/externalhit_uatext.php),173.252.95.12,-,-,200,True,scripted,18:40:36,Download,2918997772,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2457,28/04/2024,target=ijor:zijbemr&volume=4&issue=6&article=018&type=pdf,Googlebot/2.1+(+http://www.google.com/bot.html),66.249.66.160,-,-,200,True,,07:53:58,Download,1123631776,Google Scholar,First Process
2458,28/04/2024,target=ijor:zijmr&volume=2&issue=12&article=019&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",152.58.146.84,-,https://iask.ai/,200,False,,15:26:05,Download,2553975380,,
2459,28/04/2024,target=ijor:zijmr&volume=2&issue=5&article=017&type=pdf,"Mozilla/5.0+AppleWebKit/537.36+(KHTML,+like+Gecko;+compatible;+bingbot/2.0;++http://www.bing.com/bingbot.htm)+Chrome/116.0.1938.76+Safari/537.36",207.46.13.153,-,-,200,True,,23:28:08,Download,3475901849,,
2460,28/04/2024,target=ijor:zijmr&volume=2&issue=5&article=022&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",49.206.129.148,ASP.NET_SessionId=i0btfg12oiwaeo3rktcpdogw;+_gid=GA1.2.1512945033.1714279193;+_gat_gtag_UA_104381405_2=1;+_ga_RDCC5F3593=GS1.1.1714279192.1.0.1714279192.0.0.0;+_ga=GA1.1.1413538317.1714279193,https://www.indianjournals.com/ijor.aspx?target=ijor:zijmr&volume=2&issue=5&article=022,200,False,,04:39:56,Download,835617172,,


In [170]:
df_IP = pd.read_csv('iptolocation.csv')



# Define the mapping of old column names to new column names
column_mapping = {
    '0': 'start_ip',
    '16777215': 'end_ip',
    '-': 'country code ',
    '-.1': 'country',
    'Broadcast RFC1700':'ISP'
}

# Rename the columns using the mapping
df_IP.rename(columns=column_mapping, inplace=True)

start_time0 = time.time()

# Convert 'start_ip' and 'end_ip' columns to numpy arrays for faster computation
start_ips_df5 = df_IP['start_ip'].values
end_ips_df5 = df_IP['end_ip'].values
client_names_df5 = df_IP['ISP'].values
countries_df5 = df_IP['country'].values
df_downloads0 = df_downloads[df_downloads['process'] == '']
df_downloads1 = df_downloads0.copy()
# Iterate over each row in df5 to find the corresponding client name and country
for index, row in df_downloads1.iterrows():
    c_ip_decimal = row['c-ip Decimal']
    
    # Use numpy array operations to check if c-ip Decimal falls within the range of start_ip and end_ip
    mask = (start_ips_df5 <= c_ip_decimal) & (c_ip_decimal <= end_ips_df5)
    
    # If there's a match and the client name in df_downloads is empty or NaN, update it with the client name from df9
    if mask.any() : 
        df_downloads1.at[index, 'client name'] = client_names_df5[mask][0]
        df_downloads1.at[index, 'country'] = countries_df5[mask][0]
        # Add the process information as 'Third Process'
        df_downloads1.at[index, 'process'] = 'Third Process'

# Record the end time
end_time0 = time.time()

# Calculate the total execution time
execution_time0 = end_time0 - start_time0

print("Total execution time:", execution_time0, "seconds")

# Display the updated df5 with the Client Name, Country, and Process columns
df_downloads1

Total execution time: 24.898808002471924 seconds


,date,cs-uri-query,cs(User-Agent),c-ip,cs(Cookie),cs(Referer),sc-status,is_bot,scripted,time,Query_Category,c-ip Decimal,client name,process,country
0,28/04/2024,target=ijor%3Agjfsm&volume=2&issue=2&article=006&type=fulltext,"Mozilla/5.0+AppleWebKit/537.36+(KHTML,+like+Gecko;+compatible;+bingbot/2.0;++http://www.bing.com/bingbot.htm)+Chrome/116.0.1938.76+Safari/537.36",52.167.144.19,-,-,200,True,scripted,22:59:10,Download,883396627,Microsoft Corporation,Third Process,United States of America
1,28/04/2024,target=ijor%3Aijan&volume=12&issue=2&article=005&type=pdf,facebookexternalhit/1.1+(+http://www.facebook.com/externalhit_uatext.php),173.252.83.28,-,-,200,True,scripted,01:23:50,Download,2918994716,Facebook Inc.,Third Process,United States of America
2,28/04/2024,target=ijor%3Aijh&volume=31&issue=1&article=024&type=pdf,facebookexternalhit/1.1+(+http://www.facebook.com/externalhit_uatext.php),173.252.79.5,-,-,200,True,scripted,18:40:18,Download,2918993669,Facebook Inc.,Third Process,United States of America
3,28/04/2024,target=ijor%3Aijh&volume=31&issue=1&article=024&type=pdf,facebookexternalhit/1.1+(+http://www.facebook.com/externalhit_uatext.php),173.252.83.10,-,-,200,True,scripted,18:40:14,Download,2918994698,Facebook Inc.,Third Process,United States of America
4,28/04/2024,target=ijor%3Aijh&volume=31&issue=1&article=024&type=pdf,facebookexternalhit/1.1+(+http://www.facebook.com/externalhit_uatext.php),173.252.95.12,-,-,200,True,scripted,18:40:36,Download,2918997772,Facebook Inc.,Third Process,United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2439,28/04/2024,target=ijor:wea&volume=18&issue=4&article=abs122&type=pdf,"Mozilla/5.0+AppleWebKit/537.36+(KHTML,+like+Gecko;+compatible;+bingbot/2.0;++http://www.bing.com/bingbot.htm)+Chrome/116.0.1938.76+Safari/537.36",40.77.167.41,-,-,200,True,,08:49:51,Download,676177705,Microsoft Corporation,Third Process,United States of America
2440,28/04/2024,target=ijor:wea&volume=19&issue=2&article=abs123&type=pdf,Mozilla/5.0+(X11;+Linux+x86_64;+rv:103.0)+Gecko/20100101+Firefox/103.0,198.54.131.58,-,-,200,False,,07:59:43,Download,3325461306,Tzulo Inc.,Third Process,United States of America
2458,28/04/2024,target=ijor:zijmr&volume=2&issue=12&article=019&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",152.58.146.84,-,https://iask.ai/,200,False,,15:26:05,Download,2553975380,Reliance Jio Infocomm Limited,Third Process,India
2459,28/04/2024,target=ijor:zijmr&volume=2&issue=5&article=017&type=pdf,"Mozilla/5.0+AppleWebKit/537.36+(KHTML,+like+Gecko;+compatible;+bingbot/2.0;++http://www.bing.com/bingbot.htm)+Chrome/116.0.1938.76+Safari/537.36",207.46.13.153,-,-,200,True,,23:28:08,Download,3475901849,Microsoft Corporation,Third Process,United States of America


In [171]:
# List of ISP identifiers
isp_identifiers = [
    'limited', 'network', 'llc','ltd', 'telecom','telephone', 
    'company', 'service', 'telecommunications', 'communications corporation',
    'host', 'cable datacom limited', 'foundation', 'technologies', 'retail',
    'services', 'co. ltd.', 'dotcom', 'broadband', 'network internet',
    'corp.', 'inc.','inc','s.a.s.', 'plc', 'telenet', 'cellular', 'vodafone', 
    'telekom', 'wireless', 'pvt.','ltd.', 'digital', 'verizon', 'cables', 
    'skybroadband', 'communications','corporation','OVH SAS'
]

# Specific companies to flag
specific_companies = {
    'google': 'Google',
    'microsoft': 'Microsoft',
    'amazon': 'Amazon'
}


# Function to check the category of the client name
def categorize_client(client_name):
    client_name_lower = client_name.lower()
    for company, flag in specific_companies.items():
        if company in client_name_lower:
            return flag
    for isp in isp_identifiers:
        if isp in client_name_lower:
            return 'ISP'
    return 'Other'
    



In [172]:
# Drop rows from df_downloads that were identified in df_downloads1
df_downloads = df_downloads.drop(df_downloads1.index)

# Concatenate df_downloads1 with the modified df_downloads
df_downloads_final = pd.concat([df_downloads, df_downloads1], ignore_index=True)
# Apply the function to create the new column
df_downloads_final['ISP'] = df_downloads_final['client name'].apply(categorize_client)
# Optionally, you can sort the DataFrame by index
df_downloads_final.sort_index(inplace=True)
df_downloads_final

,date,cs-uri-query,cs(User-Agent),c-ip,cs(Cookie),cs(Referer),sc-status,is_bot,scripted,time,Query_Category,c-ip Decimal,client name,process,country,ISP
0,28/04/2024,target=ijor:aar1&volume=13&issue=1&article=003&type=pdf,Mozilla/5.0+(compatible;+Googlebot/2.1;++http://www.google.com/bot.html),66.249.66.169,-,-,200,True,,19:41:09,Download,1123631785,Google Scholar,First Process,NaN,Google
1,28/04/2024,target=ijor:abjmmi&volume=12&issue=2&article=012&type=pdf,Mozilla/5.0+(compatible;+Googlebot/2.1;++http://www.google.com/bot.html),66.249.66.169,-,-,200,True,,06:58:06,Download,1123631785,Google Scholar,First Process,NaN,Google
2,28/04/2024,target=ijor:abr&volume=11&issue=1and2&article=017&type=pdf,Googlebot/2.1+(+http://www.google.com/bot.html),66.249.66.169,-,-,200,True,,19:17:13,Download,1123631785,Google Scholar,First Process,NaN,Google
3,28/04/2024,target=ijor:abr&volume=23&issue=1&article=009&type=fulltext,Googlebot/2.1+(+http://www.google.com/bot.html),66.249.66.169,-,-,200,True,,01:24:18,Download,1123631785,Google Scholar,First Process,NaN,Google
4,28/04/2024,target=ijor:abr&volume=23&issue=1&article=012&type=fulltext,Googlebot/2.1+(+http://www.google.com/bot.html),66.249.66.169,-,-,200,True,,20:30:29,Download,1123631785,Google Scholar,First Process,NaN,Google
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2457,28/04/2024,target=ijor:wea&volume=18&issue=4&article=abs122&type=pdf,"Mozilla/5.0+AppleWebKit/537.36+(KHTML,+like+Gecko;+compatible;+bingbot/2.0;++http://www.bing.com/bingbot.htm)+Chrome/116.0.1938.76+Safari/537.36",40.77.167.41,-,-,200,True,,08:49:51,Download,676177705,Microsoft Corporation,Third Process,United States of America,Microsoft
2458,28/04/2024,target=ijor:wea&volume=19&issue=2&article=abs123&type=pdf,Mozilla/5.0+(X11;+Linux+x86_64;+rv:103.0)+Gecko/20100101+Firefox/103.0,198.54.131.58,-,-,200,False,,07:59:43,Download,3325461306,Tzulo Inc.,Third Process,United States of America,ISP
2459,28/04/2024,target=ijor:zijmr&volume=2&issue=12&article=019&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",152.58.146.84,-,https://iask.ai/,200,False,,15:26:05,Download,2553975380,Reliance Jio Infocomm Limited,Third Process,India,ISP
2460,28/04/2024,target=ijor:zijmr&volume=2&issue=5&article=017&type=pdf,"Mozilla/5.0+AppleWebKit/537.36+(KHTML,+like+Gecko;+compatible;+bingbot/2.0;++http://www.bing.com/bingbot.htm)+Chrome/116.0.1938.76+Safari/537.36",207.46.13.153,-,-,200,True,,23:28:08,Download,3475901849,Microsoft Corporation,Third Process,United States of America,Microsoft


In [173]:

# Get the value counts for the 'client name' column
client_name_value_counts = df_downloads_final['process'].value_counts()

# Display the value counts
print(client_name_value_counts)


process
Third Process    1826
First Process     636
Name: count, dtype: int64


In [174]:
def extract_journal_info(query):
    # Extract journal code
    journal_code_match = re.search(r'ijor:([a-zA-Z0-9]+)', query)
    journal_code = journal_code_match.group(1) if journal_code_match else None
    
    # Extract volume
    volume_match = re.search(r'volume=([a-zA-Z0-9]+)', query)
    volume = volume_match.group(1) if volume_match else None
    
    # Extract issue
    issue_match = re.search(r'issue=([a-zA-Z0-9]+)', query)
    issue = issue_match.group(1) if issue_match else None    
    return journal_code, volume, issue

df_downloads2 = df_downloads_final.copy()
df_downloads2[['Journal_Code', 'Volume', 'Issue']] = df_downloads2['cs-uri-query'].apply(lambda query: pd.Series(extract_journal_info(query)))
# Concatenate journal code, volume, and issue columns
df_downloads2['Concatenated'] = df_downloads2['Journal_Code'] + "_" + df_downloads2['Volume'].astype(str) + "_" + df_downloads2['Issue'].astype(str)
dfj=pd.read_excel('new product feed.xlsx')
# Concatenate journal code, volume, and issue columns
dfj['Concatenated'] = dfj['TITLE_ID'] + "_" + dfj['Sample Vol'].astype(str) + "_" + dfj['Sample Issue'].astype(str)
# Iterate over the rows of df6
for index, row in df_downloads2.iterrows():
    # Get the concatenated value
    concatenated_value = row['Concatenated']
    
    # Check if the concatenated value exists in df11
    if concatenated_value in dfj['Concatenated'].values:
        # If it exists, mark it as 'sample' in df6
        df_downloads2.at[index, 'Sample'] = 'sample'
    else:
        # If it doesn't exist, leave it blank
        df_downloads2.at[index, 'Sample'] = ''
# Drop the 'sample_vol_value' and 'sample_issue_value' columns
#df_downloads2.drop(columns=['Sample Vol', 'Sample Issue'], inplace=True)
df_downloads2

,date,cs-uri-query,cs(User-Agent),c-ip,cs(Cookie),cs(Referer),sc-status,is_bot,scripted,time,...,c-ip Decimal,client name,process,country,ISP,Journal_Code,Volume,Issue,Concatenated,Sample
0,28/04/2024,target=ijor:aar1&volume=13&issue=1&article=003&type=pdf,Mozilla/5.0+(compatible;+Googlebot/2.1;++http://www.google.com/bot.html),66.249.66.169,-,-,200,True,,19:41:09,...,1123631785,Google Scholar,First Process,NaN,Google,aar1,13,1,aar1_13_1,
1,28/04/2024,target=ijor:abjmmi&volume=12&issue=2&article=012&type=pdf,Mozilla/5.0+(compatible;+Googlebot/2.1;++http://www.google.com/bot.html),66.249.66.169,-,-,200,True,,06:58:06,...,1123631785,Google Scholar,First Process,NaN,Google,abjmmi,12,2,abjmmi_12_2,
2,28/04/2024,target=ijor:abr&volume=11&issue=1and2&article=017&type=pdf,Googlebot/2.1+(+http://www.google.com/bot.html),66.249.66.169,-,-,200,True,,19:17:13,...,1123631785,Google Scholar,First Process,NaN,Google,abr,11,1and2,abr_11_1and2,
3,28/04/2024,target=ijor:abr&volume=23&issue=1&article=009&type=fulltext,Googlebot/2.1+(+http://www.google.com/bot.html),66.249.66.169,-,-,200,True,,01:24:18,...,1123631785,Google Scholar,First Process,NaN,Google,abr,23,1,abr_23_1,
4,28/04/2024,target=ijor:abr&volume=23&issue=1&article=012&type=fulltext,Googlebot/2.1+(+http://www.google.com/bot.html),66.249.66.169,-,-,200,True,,20:30:29,...,1123631785,Google Scholar,First Process,NaN,Google,abr,23,1,abr_23_1,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2457,28/04/2024,target=ijor:wea&volume=18&issue=4&article=abs122&type=pdf,"Mozilla/5.0+AppleWebKit/537.36+(KHTML,+like+Gecko;+compatible;+bingbot/2.0;++http://www.bing.com/bingbot.htm)+Chrome/116.0.1938.76+Safari/537.36",40.77.167.41,-,-,200,True,,08:49:51,...,676177705,Microsoft Corporation,Third Process,United States of America,Microsoft,wea,18,4,wea_18_4,
2458,28/04/2024,target=ijor:wea&volume=19&issue=2&article=abs123&type=pdf,Mozilla/5.0+(X11;+Linux+x86_64;+rv:103.0)+Gecko/20100101+Firefox/103.0,198.54.131.58,-,-,200,False,,07:59:43,...,3325461306,Tzulo Inc.,Third Process,United States of America,ISP,wea,19,2,wea_19_2,
2459,28/04/2024,target=ijor:zijmr&volume=2&issue=12&article=019&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",152.58.146.84,-,https://iask.ai/,200,False,,15:26:05,...,2553975380,Reliance Jio Infocomm Limited,Third Process,India,ISP,zijmr,2,12,zijmr_2_12,
2460,28/04/2024,target=ijor:zijmr&volume=2&issue=5&article=017&type=pdf,"Mozilla/5.0+AppleWebKit/537.36+(KHTML,+like+Gecko;+compatible;+bingbot/2.0;++http://www.bing.com/bingbot.htm)+Chrome/116.0.1938.76+Safari/537.36",207.46.13.153,-,-,200,True,,23:28:08,...,3475901849,Microsoft Corporation,Third Process,United States of America,Microsoft,zijmr,2,5,zijmr_2_5,sample


In [175]:
df_downloads2['process'].value_counts()

process
Third Process    1826
First Process     636
Name: count, dtype: int64

In [176]:
#######################################################################
#######################################################################
# use this for calculation and viewing bots 
#######################################################################
#######################################################################
df_downloads_false = df_downloads2[df_downloads2['is_bot'] == False]
df_downloads_false 


,date,cs-uri-query,cs(User-Agent),c-ip,cs(Cookie),cs(Referer),sc-status,is_bot,scripted,time,...,c-ip Decimal,client name,process,country,ISP,Journal_Code,Volume,Issue,Concatenated,Sample
60,28/04/2024,target=ijor:ajrbem&volume=2&issue=3&article=012&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/123.0.0.0+Safari/537.36",13.234.152.96,ASP.NET_SessionId=jd32rhsrvbdivakxhcgzgypc;+_gid=GA1.2.504434268.1714283454;+_ga_RDCC5F3593=GS1.1.1714287354.2.1.1714287369.0.0.0;+_ga=GA1.1.438336251.1693833934,https://www.indianjournals.com/ijor.aspx?target=ijor:ajrbem&volume=2&issue=3&article=012,200,False,,07:16:45,...,233478240,Lal Bahadur Shastri National Academy Of Administration,First Process,NaN,Other,ajrbem,2,3,ajrbem_2_3,
87,28/04/2024,target=ijor:apps&volume=18&issue=1&article=055&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36+Edg/124.0.0.0",14.139.109.209,_ga_G5TCNFJCYP=GS1.1.1713753386.5.0.1713754370.0.0.0;+_ga=GA1.1.1438910599.1703262978;+_ga_RDCC5F3593=GS1.1.1713888060.9.0.1713888065.0.0.0,https://scholar.google.com/,200,False,,04:39:39,...,244018641,Anand Agricultural University,First Process,NaN,Other,apps,18,1,apps_18_1,
91,28/04/2024,target=ijor:ar&volume=42&issue=4&article=016&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36+Edg/124.0.0.0",14.139.41.90,_ga_G5TCNFJCYP=GS1.1.1710089998.1.0.1710089998.0.0.0;+_gid=GA1.2.1745373681.1714164112;+ASP.NET_SessionId=ayy32fcwfi2yi4eq4o5qh3s4;+_ga_RDCC5F3593=GS1.1.1714245307.3.0.1714245307.0.0.0;+_ga=GA1.2.1548072699.1710089934;+_gat_gtag_UA_104381405_2=1,https://www.indianjournals.com/ijor.aspx?target=ijor:ar&volume=42&issue=4&article=016,200,False,,19:15:22,...,244001114,Banaras Hindu University,First Process,NaN,Other,ar,42,4,ar_42_4,
145,28/04/2024,target=ijor:ija&volume=46&issue=4&article=022&type=pdf,"Mozilla/5.0+(Macintosh;+Intel+Mac+OS+X+10_15_7)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",14.139.239.22,ASP.NET_SessionId=kbjhmr1zoo12j2f10uuphmrv,https://www.indianjournals.com/ijor.aspx?target=ijor:ija&volume=46&issue=4&article=022,200,False,,08:13:52,...,244051734,Hemwati Nandan Bahuguna Garhwal University,First Process,NaN,Other,ija,46,4,ija_46_4,
146,28/04/2024,target=ijor:ija&volume=54&issue=2&article=008&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",103.55.72.34,ASP.NET_SessionId=wvv2qsvjcn3ov1tsz51pwi5k;+_gid=GA1.2.1774015368.1714244024;+_gat_gtag_UA_104381405_2=1;+_ga_G5TCNFJCYP=GS1.1.1714244421.2.0.1714244467.0.0.0;+_ga_RDCC5F3593=GS1.1.1714244024.4.1.1714244473.0.0.0;+_ga=GA1.1.235507214.1712914350,-,200,False,,19:01:22,...,1731676194,Banaras Hindu University,First Process,NaN,Other,ija,54,2,ija_54_2,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2452,28/04/2024,target=ijor:vetos&volume=31&issue=special&article=001&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/113.0.0.0+Safari/537.36",218.70.106.126,-,-,200,False,,23:45:35,...,3662047870,ChinaNet Chongqing Province Network,Third Process,China,ISP,vetos,31,special,vetos_31_special,
2456,28/04/2024,target=ijor:wea&volume=18&issue=4&article=abs074&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/112.0.0.0+Safari/537.36",45.74.6.165,-,https://www.indianjournals.com/ijor.aspx?target=ijor:wea&volume=18&issue=4&article=abs074&type=pdf,200,False,,00:56:32,...,759826085,Secure Internet LLC,Third Process,Belgium,ISP,wea,18,4,wea_18_4,
2458,28/04/2024,target=ijor:wea&volume=19&issue=2&article=abs123&type=pdf,Mozilla/5.0+(X11;+Linux+x86_64;+rv:103.0)+Gecko/20100101+Firefox/103.0,198.54.131.58,-,-,200,False,,07:59:43,...,3325461306,Tzulo Inc.,Third Process,United States of America,ISP,wea,19,2,wea

In [177]:
######################################################################################################
######################################################################################################
######################################################################################################

#           DENIALS 

######################################################################################################
######################################################################################################
######################################################################################################
import pandas as pd
from datetime import datetime, timedelta 
# Group by the specified columns and select unique 'time' values within each group
df_denial = denial.groupby(['date','cs-uri-query' ,'cs(User-Agent)','c-ip','cs(Cookie)','cs(Referer)','sc-status','is_bot','scripted'])['time'].unique().reset_index()

df_denial['time'] = df_denial['time'].apply(lambda x: [datetime.strptime(t, '%H:%M:%S') for t in x])

# Define the time window for double clicks (30 seconds)
time_window = timedelta(seconds=30)

# Initialize a list to store the indices of clicks to be removed
indices_to_remove = []

# Iterate through each row in the DataFrame
for index, row in df_denial.iterrows():
    # Get the list of time values for the current row
    times = row['time']
    
    # Iterate through consecutive pairs of time values
    for i in range(len(times) - 1):
        # Calculate the time difference between consecutive clicks
        time_diff = times[i + 1] - times[i]
        
        # Check if the time difference is within the time window
        if time_diff <= time_window:
            # Mark the later click for removal
            indices_to_remove.append(index)

# Drop the rows marked for removal
df_filtered_denial = df_denial.drop(indices_to_remove)

# Keep only the first entry for each set of consecutive clicks within the time window
df_filtered_denial
# Keep only the first entry for each set of consecutive clicks within the time window
df_filtered_denial['time'] = df_filtered_denial['time'].apply(lambda x: x[0])

# Reset the index of the filtered DataFrame
df_filtered_denial.reset_index(drop=True, inplace=True)

# Assuming df_filtered is your DataFrame
df_denials = df_filtered_denial.copy()

# Extract the time part from datetime objects
df_denials['time'] = df_denials['time'].dt.strftime('%H:%M:%S')
df_denials['Query_Category'] = 'Denial'

# Display the DataFrame with the time values extracted
def ip_to_decimal(ip):
    return int(ipaddress.IPv4Address(ip))

# Apply the function to the 'c-ip' column and create a new column 'c-ip Decimal'
df_denials['c-ip Decimal'] = df_denials['c-ip'].apply(ip_to_decimal)
df_denials

,date,cs-uri-query,cs(User-Agent),c-ip,cs(Cookie),cs(Referer),sc-status,is_bot,scripted,time,Query_Category,c-ip Decimal
0,28/04/2024,target=ijor%3Aaerr&volume=19&issue=conf&article=001&type=fulltext,Mozilla/5.0+(X11;+Linux+x86_64;+rv:103.0)+Gecko/20100101+Firefox/103.0,146.70.172.71,-,-,503,False,scripted,11:12:17,Denial,2454105159
1,28/04/2024,target=ijor%3Agjfsm&volume=2&issue=2&article=br&type=fulltext,"Mozilla/5.0+AppleWebKit/537.36+(KHTML,+like+Gecko;+compatible;+bingbot/2.0;++http://www.bing.com/bingbot.htm)+Chrome/116.0.1938.76+Safari/537.36",52.167.144.214,-,-,503,True,scripted,11:54:34,Denial,883396822
2,28/04/2024,target=ijor%3Aijan&volume=12&issue=2&article=005&type=pdf,facebookexternalhit/1.1+(+http://www.facebook.com/externalhit_uatext.php),173.252.83.28,-,-,301,True,scripted,01:23:46,Denial,2918994716
3,28/04/2024,target=ijor%3Aijh&volume=57&issue=3&article=002&type=fulltext,Mozilla/5.0+(X11;+Linux+x86_64;+rv:103.0)+Gecko/20100101+Firefox/103.0,198.54.131.58,-,-,503,False,scripted,15:16:11,Denial,3325461306
4,28/04/2024,target=ijor%3Aijh&volume=62&issue=2&article=006&type=fulltext,DoCoMo/2.0+SH901iC(c100;TB;W24H12),198.54.131.58,-,-,503,True,scripted,11:16:47,Denial,3325461306
...,...,...,...,...,...,...,...,...,...,...,...,...
11136,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=003&type=pdf,facebookexternalhit/1.1+(+http://www.facebook.com/externalhit_uatext.php),173.252.83.16,-,-,503,True,,15:44:57,Denial,2918994704
11137,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=005&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36+Edg/124.0.0.0",168.167.247.20,_ga_G5TCNFJCYP=GS1.1.1712782828.2.0.1712782828.0.0.0;+ASP.NET_SessionId=uxrfca0kmndgbyoktmw5ojax;+_ga_RDCC5F3593=GS1.1.1714253917.3.0.1714253917.0.0.0;+_ga=GA1.2.1582207726.1711665098;+_gid=GA1.2.1729083331.1714253918,https://www.indianjournals.com/ijor.aspx?target=ijor:zijmr&volume=9&issue=3&article=005,302,False,,07:51:29,Denial,2829580052
11138,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=005&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36+Edg/124.0.0.0",168.167.247.20,_ga_G5TCNFJCYP=GS1.1.1712782828.2.0.1712782828.0.0.0;+ASP.NET_SessionId=uxrfca0kmndgbyoktmw5ojax;+_gid=GA1.2.1729083331.1714253918;+_ga_RDCC5F3593=GS1.1.1714290470.4.0.1714290470.0.0.0;+_ga=GA1.1.1582207726.1711665098,https://www.indianjournals.com/ijor.aspx?target=ijor:zijmr&volume=9&issue=3&article=005,302,False,,08:30:26,Denial,2829580052
11139,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=035&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",64.224.105.15,_ga_G5TCNFJCYP=GS1.1.1713449933.2.0.1713449933.0.0.0;+ASP.NET_SessionId=5rzr3bhb4dufb1ncvyqububg;+_gid=GA1.2.588328903.1714266178;+_gat_gtag_UA_104381405_2=1;+_ga_RDCC5F3593=GS1.1.1714266177.3.0.1714266177.0.0.0;+_ga=GA1.1.757572253.1710950275,https://www.indianjournals.com/ijor.aspx?target=ijor:zijmr&volume=9&issue=3&article=035,302,False,,01:03:17,Denial,1088448783


In [178]:

import time
import pandas as pd 
df_clients=pd.read_excel('clients excel.xlsx')
df_clients
df_denials['client name'] = ''

# Record the start time
start_time = time.time()

# Convert 'Start Decimal' and 'End Decimal' columns to numpy arrays for faster computation
start_decimals_df_clients = df_clients['Start Decimal'].values
end_decimals_df_clients = df_clients['End Decimal'].values
client_names_df_clients = df_clients['client name'].values

# Initialize the 'process' column to None
df_denials['process'] = ''

# Iterate over each row in df5 to find the corresponding client name
for index, row in df_denials.iterrows():
    c_ip_decimal = row['c-ip Decimal']
    current_client_name = row['client name']
    
    # Check if the client name is blank
    if pd.isnull(current_client_name) or current_client_name.strip() == '':
        # Use numpy array operations to check if c-ip Decimal falls within the range of start and end decimals in df8
        mask = (start_decimals_df_clients <= c_ip_decimal) & (c_ip_decimal <= end_decimals_df_clients)
        
        # If there's a match, assign the corresponding client name to df5 and mark the process
        if mask.any():
            df_denials.at[index, 'client name'] = client_names_df_clients[mask][0]
            df_denials.at[index, 'process'] = 'First Process'
            
# Record the end time
end_time1 = time.time()

# Calculate the total execution time
execution_time1 = end_time1 - start_time

print("Total execution time:", execution_time1, "seconds")

# Display the updated df5 with the Client Name and Process columns
ip_mapping1.rename(columns={'Client Name': 'client name'}, inplace=True)
# Assuming df9 is your IP mapping DataFrame and df_downloads is your main DataFrame

# Create a dictionary mapping IP addresses to client names
ip_client_mapping = dict(zip(ip_mapping1['c-ip'], ip_mapping1['client name']))

# Iterate over each row in df_downloads to map IP addresses to client names and update the process column
for index, row in df_denials.iterrows():
    c_ip = row['c-ip']
    current_client_name = row['client name']
    current_process = row['process']
    
    # Check if the client name is empty or NaN
    if pd.isnull(current_client_name) or current_client_name.strip() == '':
        # Check if the IP address exists in the ip_client_mapping dictionary
        if c_ip in ip_client_mapping:
            # Assign the client name
            df_denials.at[index, 'client name'] = ip_client_mapping[c_ip]
            # Update the process column to 'process two'
            df_denials.at[index, 'process'] = 'Second Process'

# Display the DataFrame with the updated columns
df_denials

Total execution time: 0.47544336318969727 seconds


,date,cs-uri-query,cs(User-Agent),c-ip,cs(Cookie),cs(Referer),sc-status,is_bot,scripted,time,Query_Category,c-ip Decimal,client name,process
0,28/04/2024,target=ijor%3Aaerr&volume=19&issue=conf&article=001&type=fulltext,Mozilla/5.0+(X11;+Linux+x86_64;+rv:103.0)+Gecko/20100101+Firefox/103.0,146.70.172.71,-,-,503,False,scripted,11:12:17,Denial,2454105159,,
1,28/04/2024,target=ijor%3Agjfsm&volume=2&issue=2&article=br&type=fulltext,"Mozilla/5.0+AppleWebKit/537.36+(KHTML,+like+Gecko;+compatible;+bingbot/2.0;++http://www.bing.com/bingbot.htm)+Chrome/116.0.1938.76+Safari/537.36",52.167.144.214,-,-,503,True,scripted,11:54:34,Denial,883396822,,
2,28/04/2024,target=ijor%3Aijan&volume=12&issue=2&article=005&type=pdf,facebookexternalhit/1.1+(+http://www.facebook.com/externalhit_uatext.php),173.252.83.28,-,-,301,True,scripted,01:23:46,Denial,2918994716,,
3,28/04/2024,target=ijor%3Aijh&volume=57&issue=3&article=002&type=fulltext,Mozilla/5.0+(X11;+Linux+x86_64;+rv:103.0)+Gecko/20100101+Firefox/103.0,198.54.131.58,-,-,503,False,scripted,15:16:11,Denial,3325461306,,
4,28/04/2024,target=ijor%3Aijh&volume=62&issue=2&article=006&type=fulltext,DoCoMo/2.0+SH901iC(c100;TB;W24H12),198.54.131.58,-,-,503,True,scripted,11:16:47,Denial,3325461306,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11136,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=003&type=pdf,facebookexternalhit/1.1+(+http://www.facebook.com/externalhit_uatext.php),173.252.83.16,-,-,503,True,,15:44:57,Denial,2918994704,,
11137,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=005&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36+Edg/124.0.0.0",168.167.247.20,_ga_G5TCNFJCYP=GS1.1.1712782828.2.0.1712782828.0.0.0;+ASP.NET_SessionId=uxrfca0kmndgbyoktmw5ojax;+_ga_RDCC5F3593=GS1.1.1714253917.3.0.1714253917.0.0.0;+_ga=GA1.2.1582207726.1711665098;+_gid=GA1.2.1729083331.1714253918,https://www.indianjournals.com/ijor.aspx?target=ijor:zijmr&volume=9&issue=3&article=005,302,False,,07:51:29,Denial,2829580052,,
11138,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=005&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36+Edg/124.0.0.0",168.167.247.20,_ga_G5TCNFJCYP=GS1.1.1712782828.2.0.1712782828.0.0.0;+ASP.NET_SessionId=uxrfca0kmndgbyoktmw5ojax;+_gid=GA1.2.1729083331.1714253918;+_ga_RDCC5F3593=GS1.1.1714290470.4.0.1714290470.0.0.0;+_ga=GA1.1.1582207726.1711665098,https://www.indianjournals.com/ijor.aspx?target=ijor:zijmr&volume=9&issue=3&article=005,302,False,,08:30:26,Denial,2829580052,,
11139,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=035&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",64.224.105.15,_ga_G5TCNFJCYP=GS1.1.1713449933.2.0.1713449933.0.0.0;+ASP.NET_SessionId=5rzr3bhb4dufb1ncvyqububg;+_gid=GA1.2.588328903.1714266178;+_gat_gtag_UA_104381405_2=1;+_ga_RDCC5F3593=GS1.1.1714266177.3.0.1714266177.0.0.0;+_ga=GA1.1.757572253.1710950275,https://www.indianjournals.com/ijor.aspx?target=ijor:zijmr&volume=9&issue=3&article=035,302,False,,01:03:17,Denial,1088448783,,


In [179]:

start_time0 = time.time()

# Convert 'start_ip' and 'end_ip' columns to numpy arrays for faster computation
start_ips_df5 = df_IP['start_ip'].values
end_ips_df5 = df_IP['end_ip'].values
client_names_df5 = df_IP['ISP'].values
countries_df5 = df_IP['country'].values
df_denials1 = df_denials[df_denials['process'] == '']
df_denials1 = df_denials1.copy()
# Iterate over each row in df5 to find the corresponding client name and country
for index, row in df_denials1.iterrows():
    c_ip_decimal = row['c-ip Decimal']
    
    # Use numpy array operations to check if c-ip Decimal falls within the range of start_ip and end_ip
    mask = (start_ips_df5 <= c_ip_decimal) & (c_ip_decimal <= end_ips_df5)
    
    # If there's a match and the client name in df_downloads is empty or NaN, update it with the client name from df9
    if mask.any():
        df_denials1.at[index, 'client name'] = client_names_df5[mask][0]
        df_denials1.at[index, 'country'] = countries_df5[mask][0]
        # Add the process information as 'Third Process'
        df_denials1.at[index, 'process'] = 'Third Process'

# Record the end time
end_time0 = time.time()

# Calculate the total execution time
execution_time0 = end_time0 - start_time0

print("Total execution time:", execution_time0, "seconds")

# Display the updated df5 with the Client Name, Country, and Process columns
df_denials1


Total execution time: 127.89621686935425 seconds


,date,cs-uri-query,cs(User-Agent),c-ip,cs(Cookie),cs(Referer),sc-status,is_bot,scripted,time,Query_Category,c-ip Decimal,client name,process,country
0,28/04/2024,target=ijor%3Aaerr&volume=19&issue=conf&article=001&type=fulltext,Mozilla/5.0+(X11;+Linux+x86_64;+rv:103.0)+Gecko/20100101+Firefox/103.0,146.70.172.71,-,-,503,False,scripted,11:12:17,Denial,2454105159,Yorkshire Electricity,Third Process,United Kingdom of Great Britain and Northern Ireland
1,28/04/2024,target=ijor%3Agjfsm&volume=2&issue=2&article=br&type=fulltext,"Mozilla/5.0+AppleWebKit/537.36+(KHTML,+like+Gecko;+compatible;+bingbot/2.0;++http://www.bing.com/bingbot.htm)+Chrome/116.0.1938.76+Safari/537.36",52.167.144.214,-,-,503,True,scripted,11:54:34,Denial,883396822,Microsoft Corporation,Third Process,United States of America
2,28/04/2024,target=ijor%3Aijan&volume=12&issue=2&article=005&type=pdf,facebookexternalhit/1.1+(+http://www.facebook.com/externalhit_uatext.php),173.252.83.28,-,-,301,True,scripted,01:23:46,Denial,2918994716,Facebook Inc.,Third Process,United States of America
3,28/04/2024,target=ijor%3Aijh&volume=57&issue=3&article=002&type=fulltext,Mozilla/5.0+(X11;+Linux+x86_64;+rv:103.0)+Gecko/20100101+Firefox/103.0,198.54.131.58,-,-,503,False,scripted,15:16:11,Denial,3325461306,Tzulo Inc.,Third Process,United States of America
4,28/04/2024,target=ijor%3Aijh&volume=62&issue=2&article=006&type=fulltext,DoCoMo/2.0+SH901iC(c100;TB;W24H12),198.54.131.58,-,-,503,True,scripted,11:16:47,Denial,3325461306,Tzulo Inc.,Third Process,United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11136,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=003&type=pdf,facebookexternalhit/1.1+(+http://www.facebook.com/externalhit_uatext.php),173.252.83.16,-,-,503,True,,15:44:57,Denial,2918994704,Facebook Inc.,Third Process,United States of America
11137,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=005&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36+Edg/124.0.0.0",168.167.247.20,_ga_G5TCNFJCYP=GS1.1.1712782828.2.0.1712782828.0.0.0;+ASP.NET_SessionId=uxrfca0kmndgbyoktmw5ojax;+_ga_RDCC5F3593=GS1.1.1714253917.3.0.1714253917.0.0.0;+_ga=GA1.2.1582207726.1711665098;+_gid=GA1.2.1729083331.1714253918,https://www.indianjournals.com/ijor.aspx?target=ijor:zijmr&volume=9&issue=3&article=005,302,False,,07:51:29,Denial,2829580052,Botswana Telecommunications Corporation,Third Process,Botswana
11138,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=005&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36+Edg/124.0.0.0",168.167.247.20,_ga_G5TCNFJCYP=GS1.1.1712782828.2.0.1712782828.0.0.0;+ASP.NET_SessionId=uxrfca0kmndgbyoktmw5ojax;+_gid=GA1.2.1729083331.1714253918;+_ga_RDCC5F3593=GS1.1.1714290470.4.0.1714290470.0.0.0;+_ga=GA1.1.1582207726.1711665098,https://www.indianjournals.com/ijor.aspx?target=ijor:zijmr&volume=9&issue=3&article=005,302,False,,08:30:26,Denial,2829580052,Botswana Telecommunications Corporation,Third Process,Botswana
11139,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=035&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",64.224.105.15,_ga_G5TCNFJCYP=GS1.1.1713449933.2.0.1713449933.0.0.0;+ASP.NET_SessionId=5rzr3bhb4dufb1ncvyqububg;+_gid=GA1.2.588328903.1714266178;+_gat_gtag_UA_104381405_2=1;+_ga_RDCC5F3593=GS1.1.1714266177.3.0.1714266177.0.0.0;+_ga=GA1.1.757572253.1710950275,https://www.indianjournals.com/ijor.aspx?target=ijor:zijmr&volume=9&issue=3&article=035,302,False,,01:03:17,Denial,1088448783,Aptum Technologies,Third Process,United States of America


In [180]:
# Drop rows from df_downloads that were identified in df_downloads1
df_denials = df_denials.drop(df_denials1.index)

# Concatenate df_downloads1 with the modified df_downloads
df_denials_final = pd.concat([df_denials, df_denials1], ignore_index=True)
df_denials_final['ISP'] = df_denials_final['client name'].apply(categorize_client)
# Optionally, you can sort the DataFrame by index
df_denials_final.sort_index(inplace=True)
df_denials_final

,date,cs-uri-query,cs(User-Agent),c-ip,cs(Cookie),cs(Referer),sc-status,is_bot,scripted,time,Query_Category,c-ip Decimal,client name,process,country,ISP
0,28/04/2024,target=ijor:aar1&volume=6&issue=1&article=005&type=pdf,Mozilla/5.0+(compatible;+Googlebot/2.1;++http://www.google.com/bot.html),66.249.69.175,-,-,503,True,,11:52:29,Denial,1123632559,Google Scholar,First Process,NaN,Google
1,28/04/2024,target=ijor:aar1&volume=7&issue=2&article=013&type=pdf,Mozilla/5.0+(compatible;+Googlebot/2.1;++http://www.google.com/bot.html),66.249.69.162,-,-,503,True,,12:46:49,Denial,1123632546,Google Scholar,First Process,NaN,Google
2,28/04/2024,target=ijor:abr&volume=11&issue=1and2&article=010&type=pdf,Googlebot/2.1+(+http://www.google.com/bot.html),66.249.69.175,-,-,503,True,,16:33:14,Denial,1123632559,Google Scholar,First Process,NaN,Google
3,28/04/2024,target=ijor:abr&volume=21&issue=2&article=001&type=pdf,Mozilla/5.0+(compatible;+Googlebot/2.1;++http://www.google.com/bot.html),66.249.69.175,-,-,503,True,,11:59:06,Denial,1123632559,Google Scholar,First Process,NaN,Google
4,28/04/2024,target=ijor:abr&volume=23&issue=1&article=003&type=fulltext,Googlebot/2.1+(+http://www.google.com/bot.html),66.249.69.175,-,-,503,True,,13:23:35,Denial,1123632559,Google Scholar,First Process,NaN,Google
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11136,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=003&type=pdf,facebookexternalhit/1.1+(+http://www.facebook.com/externalhit_uatext.php),173.252.83.16,-,-,503,True,,15:44:57,Denial,2918994704,Facebook Inc.,Third Process,United States of America,ISP
11137,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=005&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36+Edg/124.0.0.0",168.167.247.20,_ga_G5TCNFJCYP=GS1.1.1712782828.2.0.1712782828.0.0.0;+ASP.NET_SessionId=uxrfca0kmndgbyoktmw5ojax;+_ga_RDCC5F3593=GS1.1.1714253917.3.0.1714253917.0.0.0;+_ga=GA1.2.1582207726.1711665098;+_gid=GA1.2.1729083331.1714253918,https://www.indianjournals.com/ijor.aspx?target=ijor:zijmr&volume=9&issue=3&article=005,302,False,,07:51:29,Denial,2829580052,Botswana Telecommunications Corporation,Third Process,Botswana,ISP
11138,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=005&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36+Edg/124.0.0.0",168.167.247.20,_ga_G5TCNFJCYP=GS1.1.1712782828.2.0.1712782828.0.0.0;+ASP.NET_SessionId=uxrfca0kmndgbyoktmw5ojax;+_gid=GA1.2.1729083331.1714253918;+_ga_RDCC5F3593=GS1.1.1714290470.4.0.1714290470.0.0.0;+_ga=GA1.1.1582207726.1711665098,https://www.indianjournals.com/ijor.aspx?target=ijor:zijmr&volume=9&issue=3&article=005,302,False,,08:30:26,Denial,2829580052,Botswana Telecommunications Corporation,Third Process,Botswana,ISP
11139,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=035&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",64.224.105.15,_ga_G5TCNFJCYP=GS1.1.1713449933.2.0.1713449933.0.0.0;+ASP.NET_SessionId=5rzr3bhb4dufb1ncvyqububg;+_gid=GA1.2.588328903.1714266178;+_gat_gtag_UA_104381405_2=1;+_ga_RDCC5F3593=GS1.1.1714266177.3.0.1714266177.0.0.0;+_ga=GA1.1.757572253.1710950275,https://www.indianjournals.com/ijor.aspx?target=ijor:zijmr&volume=9&issue=3&article=035,302,False,,01:03:17,Denial,1088448783,Aptum Technologies,Third Process,United States of America,ISP


In [181]:
def extract_journal_info(query):
    # Extract journal code
    journal_code_match = re.search(r'ijor:([a-zA-Z0-9]+)', query)
    journal_code = journal_code_match.group(1) if journal_code_match else None
    
    # Extract volume
    volume_match = re.search(r'volume=([a-zA-Z0-9]+)', query)
    volume = volume_match.group(1) if volume_match else None
    
    # Extract issue
    issue_match = re.search(r'issue=([a-zA-Z0-9]+)', query)
    issue = issue_match.group(1) if issue_match else None    
    return journal_code, volume, issue
df_denials2=df_denials_final.copy()
df_denials2[['Journal_Code', 'Volume', 'Issue']] = df_denials2['cs-uri-query'].apply(lambda query: pd.Series(extract_journal_info(query)))
# Concatenate journal code, volume, and issue columns
df_denials2['Concatenated'] = df_denials2['Journal_Code'] + "_" + df_denials2['Volume'].astype(str) + "_" + df_denials2['Issue'].astype(str)

# Iterate over the rows of df6
for index, row in df_denials2.iterrows():
    # Get the concatenated value
    concatenated_value = row['Concatenated']
    
    # Check if the concatenated value exists in df11
    if concatenated_value in dfj['Concatenated'].values:
        # If it exists, mark it as 'sample' in df6
        df_denials2.at[index, 'Sample'] = 'sample'
    else:
        # If it doesn't exist, leave it blank
        df_denials2.at[index, 'Sample'] = ''
# Drop the 'sample_vol_value' and 'sample_issue_value' columns
#df_downloads2.drop(columns=['Sample Vol', 'Sample Issue'], inplace=True)
df_denials2

,date,cs-uri-query,cs(User-Agent),c-ip,cs(Cookie),cs(Referer),sc-status,is_bot,scripted,time,...,c-ip Decimal,client name,process,country,ISP,Journal_Code,Volume,Issue,Concatenated,Sample
0,28/04/2024,target=ijor:aar1&volume=6&issue=1&article=005&type=pdf,Mozilla/5.0+(compatible;+Googlebot/2.1;++http://www.google.com/bot.html),66.249.69.175,-,-,503,True,,11:52:29,...,1123632559,Google Scholar,First Process,NaN,Google,aar1,6,1,aar1_6_1,
1,28/04/2024,target=ijor:aar1&volume=7&issue=2&article=013&type=pdf,Mozilla/5.0+(compatible;+Googlebot/2.1;++http://www.google.com/bot.html),66.249.69.162,-,-,503,True,,12:46:49,...,1123632546,Google Scholar,First Process,NaN,Google,aar1,7,2,aar1_7_2,
2,28/04/2024,target=ijor:abr&volume=11&issue=1and2&article=010&type=pdf,Googlebot/2.1+(+http://www.google.com/bot.html),66.249.69.175,-,-,503,True,,16:33:14,...,1123632559,Google Scholar,First Process,NaN,Google,abr,11,1and2,abr_11_1and2,
3,28/04/2024,target=ijor:abr&volume=21&issue=2&article=001&type=pdf,Mozilla/5.0+(compatible;+Googlebot/2.1;++http://www.google.com/bot.html),66.249.69.175,-,-,503,True,,11:59:06,...,1123632559,Google Scholar,First Process,NaN,Google,abr,21,2,abr_21_2,
4,28/04/2024,target=ijor:abr&volume=23&issue=1&article=003&type=fulltext,Googlebot/2.1+(+http://www.google.com/bot.html),66.249.69.175,-,-,503,True,,13:23:35,...,1123632559,Google Scholar,First Process,NaN,Google,abr,23,1,abr_23_1,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11136,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=003&type=pdf,facebookexternalhit/1.1+(+http://www.facebook.com/externalhit_uatext.php),173.252.83.16,-,-,503,True,,15:44:57,...,2918994704,Facebook Inc.,Third Process,United States of America,ISP,zijmr,9,3,zijmr_9_3,
11137,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=005&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36+Edg/124.0.0.0",168.167.247.20,_ga_G5TCNFJCYP=GS1.1.1712782828.2.0.1712782828.0.0.0;+ASP.NET_SessionId=uxrfca0kmndgbyoktmw5ojax;+_ga_RDCC5F3593=GS1.1.1714253917.3.0.1714253917.0.0.0;+_ga=GA1.2.1582207726.1711665098;+_gid=GA1.2.1729083331.1714253918,https://www.indianjournals.com/ijor.aspx?target=ijor:zijmr&volume=9&issue=3&article=005,302,False,,07:51:29,...,2829580052,Botswana Telecommunications Corporation,Third Process,Botswana,ISP,zijmr,9,3,zijmr_9_3,
11138,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=005&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36+Edg/124.0.0.0",168.167.247.20,_ga_G5TCNFJCYP=GS1.1.1712782828.2.0.1712782828.0.0.0;+ASP.NET_SessionId=uxrfca0kmndgbyoktmw5ojax;+_gid=GA1.2.1729083331.1714253918;+_ga_RDCC5F3593=GS1.1.1714290470.4.0.1714290470.0.0.0;+_ga=GA1.1.1582207726.1711665098,https://www.indianjournals.com/ijor.aspx?target=ijor:zijmr&volume=9&issue=3&article=005,302,False,,08:30:26,...,2829580052,Botswana Telecommunications Corporation,Third Process,Botswana,ISP,zijmr,9,3,zijmr_9_3,
11139,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=035&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",64.224.105.15,_ga_G5TCNFJCYP=GS1.1.1713449933.2.0.1713449933.0.0.0;+ASP.NET_SessionId=5rzr3bhb4dufb1ncvyqububg;+_gid=GA1.2.588328903.1714266178;+_gat_gtag_UA_104381405_2=1;+_ga_RDCC5F3593=GS1.1.1714266177.3.0.1714266177.0.0.0;+_ga=GA1.1.757572253.1710950275,https://www.indianjournals.com/ijor.aspx?target=ijor:zijmr&volume=9&issue=3&article=035,302,False,,01:03:17,...,1088448783,Aptum Technologies,Third Process,United States of America,ISP,zijmr,9,3,zijmr_9_3,


In [182]:
# Concatenate df_denials2 and df_downloads2
df_downloads_and_denials = pd.concat([df_denials2, df_downloads2], ignore_index=True)

# Optionally, you can sort the DataFrame by index
df_downloads_and_denials.sort_index(inplace=True)
df_downloads_and_denials

,date,cs-uri-query,cs(User-Agent),c-ip,cs(Cookie),cs(Referer),sc-status,is_bot,scripted,time,...,c-ip Decimal,client name,process,country,ISP,Journal_Code,Volume,Issue,Concatenated,Sample
0,28/04/2024,target=ijor:aar1&volume=6&issue=1&article=005&type=pdf,Mozilla/5.0+(compatible;+Googlebot/2.1;++http://www.google.com/bot.html),66.249.69.175,-,-,503,True,,11:52:29,...,1123632559,Google Scholar,First Process,NaN,Google,aar1,6,1,aar1_6_1,
1,28/04/2024,target=ijor:aar1&volume=7&issue=2&article=013&type=pdf,Mozilla/5.0+(compatible;+Googlebot/2.1;++http://www.google.com/bot.html),66.249.69.162,-,-,503,True,,12:46:49,...,1123632546,Google Scholar,First Process,NaN,Google,aar1,7,2,aar1_7_2,
2,28/04/2024,target=ijor:abr&volume=11&issue=1and2&article=010&type=pdf,Googlebot/2.1+(+http://www.google.com/bot.html),66.249.69.175,-,-,503,True,,16:33:14,...,1123632559,Google Scholar,First Process,NaN,Google,abr,11,1and2,abr_11_1and2,
3,28/04/2024,target=ijor:abr&volume=21&issue=2&article=001&type=pdf,Mozilla/5.0+(compatible;+Googlebot/2.1;++http://www.google.com/bot.html),66.249.69.175,-,-,503,True,,11:59:06,...,1123632559,Google Scholar,First Process,NaN,Google,abr,21,2,abr_21_2,
4,28/04/2024,target=ijor:abr&volume=23&issue=1&article=003&type=fulltext,Googlebot/2.1+(+http://www.google.com/bot.html),66.249.69.175,-,-,503,True,,13:23:35,...,1123632559,Google Scholar,First Process,NaN,Google,abr,23,1,abr_23_1,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13598,28/04/2024,target=ijor:wea&volume=18&issue=4&article=abs122&type=pdf,"Mozilla/5.0+AppleWebKit/537.36+(KHTML,+like+Gecko;+compatible;+bingbot/2.0;++http://www.bing.com/bingbot.htm)+Chrome/116.0.1938.76+Safari/537.36",40.77.167.41,-,-,200,True,,08:49:51,...,676177705,Microsoft Corporation,Third Process,United States of America,Microsoft,wea,18,4,wea_18_4,
13599,28/04/2024,target=ijor:wea&volume=19&issue=2&article=abs123&type=pdf,Mozilla/5.0+(X11;+Linux+x86_64;+rv:103.0)+Gecko/20100101+Firefox/103.0,198.54.131.58,-,-,200,False,,07:59:43,...,3325461306,Tzulo Inc.,Third Process,United States of America,ISP,wea,19,2,wea_19_2,
13600,28/04/2024,target=ijor:zijmr&volume=2&issue=12&article=019&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",152.58.146.84,-,https://iask.ai/,200,False,,15:26:05,...,2553975380,Reliance Jio Infocomm Limited,Third Process,India,ISP,zijmr,2,12,zijmr_2_12,
13601,28/04/2024,target=ijor:zijmr&volume=2&issue=5&article=017&type=pdf,"Mozilla/5.0+AppleWebKit/537.36+(KHTML,+like+Gecko;+compatible;+bingbot/2.0;++http://www.bing.com/bingbot.htm)+Chrome/116.0.1938.76+Safari/537.36",207.46.13.153,-,-,200,True,,23:28:08,...,3475901849,Microsoft Corporation,Third Process,United States of America,Microsoft,zijmr,2,5,zijmr_2_5,sample


In [183]:
journal_mapping = dict(zip(dfj['PROPRIETRY_ID'], zip(dfj['TITLE'], dfj['Access Type'], dfj['isActive'])))

journal_mapping

# Define a function to map journal codes and titles using the journal_mapping dictionary
def map_journal_info(query):
    for key, value in journal_mapping.items():
        if key in query:
            return key, *value
    return 'Unknown', 'Unknown', 'Unknown', 'Unknown'

# Apply the function to create new columns 'journal_code', 'journal_title', 'Access Type', and 'isActive'
df_downloads_and_denials[['journal_code', 'journal_title', 'Access Type', 'isActive']] = df_downloads_and_denials['cs-uri-query'].apply(lambda x: pd.Series(map_journal_info(x)))

In [184]:
# process downloads with bots
df_downloads_First_Process = df_downloads[df_downloads['process'] == 'First Process']
df_downloads_Second_Process = df_downloads[df_downloads['process'] == 'Second Process']
df_downloads_Third_Process = df_downloads1[df_downloads1['process'] == 'Third Process']
# process downloads without bots
df_downloads_no_bots = df_downloads2[df_downloads2['is_bot'] == False]
df_downloads_first_process_no_bots = df_downloads_no_bots[df_downloads_no_bots['process'] == 'First Process']
df_downloads_second_process_no_bots = df_downloads_no_bots[df_downloads_no_bots['process'] == 'Second Process']
df_downloads_third_process_no_bots = df_downloads_no_bots[df_downloads_no_bots['process'] == 'Third Process']
# process denials with bots
df_denials_Third_Process = df_denials1[df_denials1['process'] == 'Third Process']
df_denials_Second_Process = df_denials2[df_denials2['process'] == 'Second Process']
df_denials_First_Process = df_denials_final[df_denials_final['process'] == 'First Process']
# process denials without bots
df_denials_no_bots = df_denials2[df_denials2['is_bot'] == False]
df_denials_first_process_no_bots = df_denials_no_bots[df_denials_no_bots['process'] == 'First Process']
df_denials_second_process_no_bots = df_denials_no_bots[df_denials_no_bots['process'] == 'Second Process']
df_denials_third_process_no_bots = df_denials_no_bots[df_denials_no_bots['process'] == 'Third Process']

# calculations of without bots from investigations 
successful_investigation_queries_without_bots = successful_investigation_queries[successful_investigation_queries['is_bot'] == False]
unsuccessful_investigation_queries_without_bots = unsuccessful_investigation_queries[unsuccessful_investigation_queries['is_bot'] == False]


# applying journal mapping
df_downloads_third_process_no_botsx=df_downloads_third_process_no_bots.copy()
df_downloads_third_process_no_botsx[['journal_code', 'journal_title', 'Access Type', 'isActive']] = df_downloads_third_process_no_botsx['cs-uri-query'].apply(lambda x: pd.Series(map_journal_info(x)))
df_denials_third_process_no_botsx=df_denials_third_process_no_bots.copy()
df_denials_third_process_no_botsx[['journal_code', 'journal_title', 'Access Type', 'isActive']] = df_denials_third_process_no_botsx['cs-uri-query'].apply(lambda x: pd.Series(map_journal_info(x)))


#access types categorization (downloads)
paid_access_downloads = df_downloads_third_process_no_botsx[df_downloads_third_process_no_botsx['Access Type'] == 'Paid']
Unknown_access_downloads = df_downloads_third_process_no_botsx[df_downloads_third_process_no_botsx['Access Type'] == 'Unknown']
Hybrid_access_downloads = df_downloads_third_process_no_botsx[df_downloads_third_process_no_botsx['Access Type'] == 'Hybrid']

#access types categorization (denials) 
paid_access_denials = df_denials_third_process_no_botsx[df_denials_third_process_no_botsx['Access Type'] == 'Paid']
Unknown_access_denials = df_denials_third_process_no_botsx[df_denials_third_process_no_botsx['Access Type'] == 'Unknown']
Hybrid_access_denials = df_denials_third_process_no_botsx[df_denials_third_process_no_botsx['Access Type'] == 'Hybrid']

#access types categorization (isActive)

active_downloads = df_downloads_third_process_no_botsx[df_downloads_third_process_no_botsx['isActive'] == 'Active']
unknown_downloads = df_downloads_third_process_no_botsx[df_downloads_third_process_no_botsx['isActive'] == 'Unknown']

#access types categorization (denials) 
active_denials = df_denials_third_process_no_botsx[df_denials_third_process_no_botsx['isActive'] == 'Active']
Unknown_denials = df_denials_third_process_no_botsx[df_denials_third_process_no_botsx['isActive'] == 'Unknown']

In [185]:
import pandas as pd
#unique downloads': df_filtered
# Create a dictionary with the names of the rows and the corresponding DataFrames
data = {
    'total entries': df1,
    'total bots': df_total_bot,
    'downloads (categorization)': download_queries,
    'total downloads (after double click filter)': df_downloads_final,
    'First process (client list)': df_downloads_First_Process,
    'Second process (Nlist)': df_downloads_Second_Process,
    'Third Process (IP to location)': df_downloads_Third_Process,
    'total downloads without bots': df_downloads_no_bots,
    'First process (client list) (no bots)': df_downloads_first_process_no_bots,
    'Second process (Nlist) (no bots)': df_downloads_second_process_no_bots,
    'Third Process (IP to location) (no bots)': df_downloads_third_process_no_bots,
    '3rd process without bots paid access downloads': paid_access_downloads,
    '3rd process without bots Unknown access downloads': Unknown_access_downloads,
    '3rd process without bots Hybrid access downloads': Hybrid_access_downloads,
    '3rd process without bots active downloads': active_downloads,
    '3rd process without bots unknown downloads': unknown_downloads,
    'denials (categorization)': denial,
    'total denials (after double click filter)': df_denials,
    'First process (client list) (denials)': df_denials_First_Process,
    'Second process (Nlist) (denials)': df_denials_Second_Process,
    'Third Process (IP to location) (denials)': df_denials_Third_Process,
    'total denials without bots': df_denials_no_bots,
    'First process (client list) (no bots) (denials)': df_denials_first_process_no_bots,
    'Second process (Nlist) (no bots) (denials)': df_denials_second_process_no_bots,
    'Third Process (IP to location) (no bots) (denials)': df_denials_third_process_no_bots,
    '3rd process without bots paid access denials': paid_access_denials,
    '3rd process without bots Unknown access denials': Unknown_access_denials,
    '3rd process without bots Hybrid access denials': Hybrid_access_denials,
    '3rd process without bots active denials': active_denials,
    '3rd process without bots Unknown denials': Unknown_denials,
    'successful investigation queries' : successful_investigation_queries,
    'successful investigation queries without bots' : successful_investigation_queries_without_bots,
    'unsuccessful investigation queries': unsuccessful_investigation_queries,
    'unsuccessful investigation queries without bots' : unsuccessful_investigation_queries_without_bots
}

# Create an empty list to store the rows for the summary DataFrame
summary_rows = []

# Loop through the dictionary and calculate the length of each DataFrame
for name, df in data.items():
    total_entries = len(df)
    summary_rows.append({'Name': name, 'Total Entries': total_entries})
pd.set_option('display.max_colwidth', None)
# Create the summary DataFrame from the list of rows
summary_df = pd.DataFrame(summary_rows)

# Display the summary DataFrame
summary_df

,Name,Total Entries
0,total entries,606848
1,total bots,116943
2,downloads (categorization),2679
3,total downloads (after double click filter),2462
4,First process (client list),636
5,Second process (Nlist),0
6,Third Process (IP to location),1826
7,total downloads without bots,632
8,First process (client list) (no bots),33
9,Second process (Nlist) (no bots),0


In [186]:
df_downloads_and_denials.columns

Index(['date', 'cs-uri-query', 'cs(User-Agent)', 'c-ip', 'cs(Cookie)',
       'cs(Referer)', 'sc-status', 'is_bot', 'scripted', 'time',
       'Query_Category', 'c-ip Decimal', 'client name', 'process', 'country',
       'ISP', 'Journal_Code', 'Volume', 'Issue', 'Concatenated', 'Sample',
       'journal_code', 'journal_title', 'Access Type', 'isActive'],
      dtype='object')

In [187]:
df_downloads_and_denials

,date,cs-uri-query,cs(User-Agent),c-ip,cs(Cookie),cs(Referer),sc-status,is_bot,scripted,time,...,ISP,Journal_Code,Volume,Issue,Concatenated,Sample,journal_code,journal_title,Access Type,isActive
0,28/04/2024,target=ijor:aar1&volume=6&issue=1&article=005&type=pdf,Mozilla/5.0+(compatible;+Googlebot/2.1;++http://www.google.com/bot.html),66.249.69.175,-,-,503,True,,11:52:29,...,Google,aar1,6,1,aar1_6_1,,ijor:aar1,Advances in Applied Research,Paid,Active
1,28/04/2024,target=ijor:aar1&volume=7&issue=2&article=013&type=pdf,Mozilla/5.0+(compatible;+Googlebot/2.1;++http://www.google.com/bot.html),66.249.69.162,-,-,503,True,,12:46:49,...,Google,aar1,7,2,aar1_7_2,,ijor:aar1,Advances in Applied Research,Paid,Active
2,28/04/2024,target=ijor:abr&volume=11&issue=1and2&article=010&type=pdf,Googlebot/2.1+(+http://www.google.com/bot.html),66.249.69.175,-,-,503,True,,16:33:14,...,Google,abr,11,1and2,abr_11_1and2,,ijor:abr,Applied Biological Research,Paid,Active
3,28/04/2024,target=ijor:abr&volume=21&issue=2&article=001&type=pdf,Mozilla/5.0+(compatible;+Googlebot/2.1;++http://www.google.com/bot.html),66.249.69.175,-,-,503,True,,11:59:06,...,Google,abr,21,2,abr_21_2,,ijor:abr,Applied Biological Research,Paid,Active
4,28/04/2024,target=ijor:abr&volume=23&issue=1&article=003&type=fulltext,Googlebot/2.1+(+http://www.google.com/bot.html),66.249.69.175,-,-,503,True,,13:23:35,...,Google,abr,23,1,abr_23_1,,ijor:abr,Applied Biological Research,Paid,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13598,28/04/2024,target=ijor:wea&volume=18&issue=4&article=abs122&type=pdf,"Mozilla/5.0+AppleWebKit/537.36+(KHTML,+like+Gecko;+compatible;+bingbot/2.0;++http://www.bing.com/bingbot.htm)+Chrome/116.0.1938.76+Safari/537.36",40.77.167.41,-,-,200,True,,08:49:51,...,Microsoft,wea,18,4,wea_18_4,,Unknown,Unknown,Unknown,Unknown
13599,28/04/2024,target=ijor:wea&volume=19&issue=2&article=abs123&type=pdf,Mozilla/5.0+(X11;+Linux+x86_64;+rv:103.0)+Gecko/20100101+Firefox/103.0,198.54.131.58,-,-,200,False,,07:59:43,...,ISP,wea,19,2,wea_19_2,,Unknown,Unknown,Unknown,Unknown
13600,28/04/2024,target=ijor:zijmr&volume=2&issue=12&article=019&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",152.58.146.84,-,https://iask.ai/,200,False,,15:26:05,...,ISP,zijmr,2,12,zijmr_2_12,,ijor:zijmr,ZENITH International Journal of Multidisciplinary Research,Paid,Active
13601,28/04/2024,target=ijor:zijmr&volume=2&issue=5&article=017&type=pdf,"Mozilla/5.0+AppleWebKit/537.36+(KHTML,+like+Gecko;+compatible;+bingbot/2.0;++http://www.bing.com/bingbot.htm)+Chrome/116.0.1938.76+Safari/537.36",207.46.13.153,-,-,200,True,,23:28:08,...,Microsoft,zijmr,2,5,zijmr_2_5,sample,ijor:zijmr,ZENITH International Journal of Multidisciplinary Research,Paid,Active


In [188]:
df_downloads_first_process_no_bots = df_downloads_no_bots[df_downloads_no_bots['process'] == 'First Process']
df_downloads_second_process_no_bots = df_downloads_no_bots[df_downloads_no_bots['process'] == 'Second Process']
df_downloads_third_process_no_bots = df_downloads_no_bots[df_downloads_no_bots['process'] == 'Third Process']
# Concatenate df_downloads1 with the modified df_downloads
df_downloads_first_second_process_no_bots = pd.concat([df_downloads_first_process_no_bots, df_downloads_second_process_no_bots], ignore_index=True)

# Optionally, you can sort the DataFrame by index
df_downloads_first_second_process_no_bots.sort_index(inplace=True)
df_downloads_first_second_process_no_bots

,date,cs-uri-query,cs(User-Agent),c-ip,cs(Cookie),cs(Referer),sc-status,is_bot,scripted,time,...,c-ip Decimal,client name,process,country,ISP,Journal_Code,Volume,Issue,Concatenated,Sample
0,28/04/2024,target=ijor:ajrbem&volume=2&issue=3&article=012&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/123.0.0.0+Safari/537.36",13.234.152.96,ASP.NET_SessionId=jd32rhsrvbdivakxhcgzgypc;+_gid=GA1.2.504434268.1714283454;+_ga_RDCC5F3593=GS1.1.1714287354.2.1.1714287369.0.0.0;+_ga=GA1.1.438336251.1693833934,https://www.indianjournals.com/ijor.aspx?target=ijor:ajrbem&volume=2&issue=3&article=012,200,False,,07:16:45,...,233478240,Lal Bahadur Shastri National Academy Of Administration,First Process,NaN,Other,ajrbem,2,3,ajrbem_2_3,
1,28/04/2024,target=ijor:apps&volume=18&issue=1&article=055&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36+Edg/124.0.0.0",14.139.109.209,_ga_G5TCNFJCYP=GS1.1.1713753386.5.0.1713754370.0.0.0;+_ga=GA1.1.1438910599.1703262978;+_ga_RDCC5F3593=GS1.1.1713888060.9.0.1713888065.0.0.0,https://scholar.google.com/,200,False,,04:39:39,...,244018641,Anand Agricultural University,First Process,NaN,Other,apps,18,1,apps_18_1,
2,28/04/2024,target=ijor:ar&volume=42&issue=4&article=016&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36+Edg/124.0.0.0",14.139.41.90,_ga_G5TCNFJCYP=GS1.1.1710089998.1.0.1710089998.0.0.0;+_gid=GA1.2.1745373681.1714164112;+ASP.NET_SessionId=ayy32fcwfi2yi4eq4o5qh3s4;+_ga_RDCC5F3593=GS1.1.1714245307.3.0.1714245307.0.0.0;+_ga=GA1.2.1548072699.1710089934;+_gat_gtag_UA_104381405_2=1,https://www.indianjournals.com/ijor.aspx?target=ijor:ar&volume=42&issue=4&article=016,200,False,,19:15:22,...,244001114,Banaras Hindu University,First Process,NaN,Other,ar,42,4,ar_42_4,
3,28/04/2024,target=ijor:ija&volume=46&issue=4&article=022&type=pdf,"Mozilla/5.0+(Macintosh;+Intel+Mac+OS+X+10_15_7)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",14.139.239.22,ASP.NET_SessionId=kbjhmr1zoo12j2f10uuphmrv,https://www.indianjournals.com/ijor.aspx?target=ijor:ija&volume=46&issue=4&article=022,200,False,,08:13:52,...,244051734,Hemwati Nandan Bahuguna Garhwal University,First Process,NaN,Other,ija,46,4,ija_46_4,
4,28/04/2024,target=ijor:ija&volume=54&issue=2&article=008&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",103.55.72.34,ASP.NET_SessionId=wvv2qsvjcn3ov1tsz51pwi5k;+_gid=GA1.2.1774015368.1714244024;+_gat_gtag_UA_104381405_2=1;+_ga_G5TCNFJCYP=GS1.1.1714244421.2.0.1714244467.0.0.0;+_ga_RDCC5F3593=GS1.1.1714244024.4.1.1714244473.0.0.0;+_ga=GA1.1.235507214.1712914350,-,200,False,,19:01:22,...,1731676194,Banaras Hindu University,First Process,NaN,Other,ija,54,2,ija_54_2,
5,28/04/2024,target=ijor:ija&volume=54&issue=2&article=008&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",103.55.72.34,ASP.NET_SessionId=wvv2qsvjcn3ov1tsz51pwi5k;+_gid=GA1.2.1774015368.1714244024;+_gat_gtag_UA_104381405_2=1;+_ga_G5TCNFJCYP=GS1.1.1714244421.2.0.1714244467.0.0.0;+_ga_RDCC5F3593=GS1.1.1714244024.4.1.1714244473.0.0.0;+_ga=GA1.1.235507214.1712914350,https://www.indianjournals.com/ijor.aspx?target=ijor:ija&volume=54&issue=2&article=008,200,False,,19:01:21,...,1731676194,Banaras Hindu University,First Process,NaN,Other,ija,54,2,ija_54_2,
6,28/04/2024,target=ijor:ija&volume=57&issue=3s&article=015&type=pdf,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",103.55.72.34,ASP.NET_SessionId=wvv2qsvjcn3ov1tsz51pwi5k;+_gid=GA1.2.1774015368.1714244024;+_ga_G5TCNFJCYP=GS1.1.1714244421.2.0.1714244421.0.0.0;+_gat_gtag_UA_104381405_2=1;+_ga_RDCC5F3593=GS1.1.1714244024.4.1.1714244431.0.0.0;+_ga=GA1.1.235507214.1712914350,-,200,False,,19:00:46,...,1731

In [189]:
#df_bots
#df_downloads_first_second_process_downloads
#df_downloads_first_second_process_denials
#df_downloads_third_process_downloads
#df_downloads_third_process_denials

In [190]:
df_bots = df_downloads_and_denials[df_downloads_and_denials['is_bot'] == True]

In [191]:
#df_downloads_first_second_process

In [192]:
######################################################################################################
######################################################################################################
######################################################################################################

#         Investigations 

######################################################################################################
######################################################################################################
######################################################################################################
import pandas as pd
import ipaddress
from datetime import datetime, timedelta 
# Group by the specified columns and select unique 'time' values within each group
successful_investigation_queries = successful_investigation_queries.groupby(['date','cs-uri-query' ,'cs(User-Agent)','c-ip','cs(Cookie)','cs(Referer)','sc-status','is_bot','scripted'])['time'].unique().reset_index()

successful_investigation_queries['time'] = successful_investigation_queries['time'].apply(lambda x: [datetime.strptime(t, '%H:%M:%S') for t in x])


# Define the time window for double clicks (30 seconds)
time_window = timedelta(seconds=30)

# Initialize a list to store the indices of clicks to be removed
indices_to_remove = []

# Iterate through each row in the DataFrame
for index, row in successful_investigation_queries.iterrows():
    # Get the list of time values for the current row
    times = row['time']
    
    # Iterate through consecutive pairs of time values
    for i in range(len(times) - 1):
        # Calculate the time difference between consecutive clicks
        time_diff = times[i + 1] - times[i]
        
        # Check if the time difference is within the time window
        if time_diff <= time_window:
            # Mark the later click for removal
            indices_to_remove.append(index)

# Drop the rows marked for removal
df_filtered_investigations = successful_investigation_queries.drop(indices_to_remove)

# Keep only the first entry for each set of consecutive clicks within the time window
df_filtered_investigations['time'] = df_filtered_investigations['time'].apply(lambda x: x[0])

# Reset the index of the filtered DataFrame
df_filtered_investigations.reset_index(drop=True, inplace=True)

# Assuming df_filtered is your DataFrame
df_investigations1 = df_filtered_investigations.copy()

# Extract the time part from datetime objects
df_investigations1['time'] = df_investigations1['time'].dt.strftime('%H:%M:%S')
df_investigations1['Query_Category'] = 'Successful Investigation'
# Display the DataFrame with the time values extracted
# df_downloads ( view this )

def ip_to_decimal(ip):
    return int(ipaddress.IPv4Address(ip))

# Apply the function to the 'c-ip' column and create a new column 'c-ip Decimal'
df_investigations1['c-ip Decimal'] = df_investigations1['c-ip'].apply(ip_to_decimal)
df_investigations1

,date,cs-uri-query,cs(User-Agent),c-ip,cs(Cookie),cs(Referer),sc-status,is_bot,scripted,time,Query_Category,c-ip Decimal
0,28/04/2024,Target=ijor%3aIjanm&volume=4&issue=1&article=016,facebookexternalhit/1.1+(+http://www.facebook.com/externalhit_uatext.php),173.252.83.8,-,-,200,True,scripted,23:29:00,Successful Investigation,2918994696
1,28/04/2024,article=001%25%21%28EXTRA%2F%2A%2A%2Fstring%3D%22%29%29%29%2C%2F%2A%2A%2Fstring%3D%28SELECT%2F%2A%2A%2FCONCAT%280x635a4356%2C%28NULL%29%2C0x42387231%29%29%2C%2F%2A%2A%2Fstring%3D%2C%2F%2A%2A%2Fstring%3D%2F%2A%2A%2FAND%2F%2A%2A%2F%28%28%28%22pOes%22%2F%2A%2A%2FLIKE%2F%2A%2A%2F%22pOes%29&issue=10&target=ijor%3Aijpss&volume=4,Wget/1.12+(freebsd8.1),198.54.131.58,-,-,200,True,scripted,20:10:19,Successful Investigation,3325461306
2,28/04/2024,article=001%25%21%28EXTRA%2F%2A%2A%2Fstring%3D%22%29%29%29%2C%2F%2A%2A%2Fstring%3D%28SELECT%2F%2A%2A%2FCONCAT%280x70566839%2C%28SELECT%2F%2A%2A%2F%28ELT%282836%3D2836%2C1%29%29%29%2C0x30323176%29%29%2C%2F%2A%2A%2Fstring%3D%2C%2F%2A%2A%2Fstring%3D%2F%2A%2A%2FAND%2F%2A%2A%2F%28%28%28%22asRP%22%2F%2A%2A%2FLIKE%2F%2A%2A%2F%22asRP%29&issue=10&target=ijor%3Aijpss&volume=4,Wget/1.12+(freebsd8.1),198.54.131.58,-,-,200,True,scripted,20:11:56,Successful Investigation,3325461306
3,28/04/2024,article=001%25%21%28EXTRA%2F%2A%2A%2Fstring%3D%22%29%29%2C%2F%2A%2A%2Fstring%3D%28SELECT%2F%2A%2A%2FCONCAT%280x336d5673%2C%28SELECT%2F%2A%2A%2F%28ELT%282836%3D2836%2C1%29%29%29%2C0x79336578%29%29%2C%2F%2A%2A%2Fstring%3D%2C%2F%2A%2A%2Fstring%3D%2F%2A%2A%2FAND%2F%2A%2A%2F%28%28%22Zgqt%22%2F%2A%2A%2FLIKE%2F%2A%2A%2F%22Zgqt%29&issue=10&target=ijor%3Aijpss&volume=4,Wget/1.12+(freebsd8.1),198.54.131.58,-,-,200,True,scripted,20:00:05,Successful Investigation,3325461306
4,28/04/2024,article=001%25%21%28EXTRA%2F%2A%2A%2Fstring%3D%22%29%29%2C%2F%2A%2A%2Fstring%3D%28SELECT%2F%2A%2A%2FCONCAT%280x74426f58%2C%28NULL%29%2C0x44586955%29%29%2C%2F%2A%2A%2Fstring%3D%2C%2F%2A%2A%2Fstring%3D%2F%2A%2A%2FAND%2F%2A%2A%2F%28%28%220TAd%22%2F%2A%2A%2FLIKE%2F%2A%2A%2F%220TAd%29&issue=10&target=ijor%3Aijpss&volume=4,Wget/1.12+(freebsd8.1),198.54.131.58,-,-,200,True,scripted,19:59:59,Successful Investigation,3325461306
...,...,...,...,...,...,...,...,...,...,...,...,...
113830,28/04/2024,volume=77&issue=4&article=010&target=ijor:ije,"Mozilla/5.0+(Macintosh;+Intel+Mac+OS+X+10_15_5)+AppleWebKit/605.1.15+(KHTML,+like+Gecko)+Version/13.1.1+Safari/605.1.15+(Applebot/0.1;++http://www.apple.com/go/applebot)",17.241.227.34,-,-,200,True,,20:18:30,Successful Investigation,301064994
113831,28/04/2024,volume=8&issue=12&article=018&target=ijor:rjpt,"Mozilla/5.0+(Macintosh;+Intel+Mac+OS+X+10_15_5)+AppleWebKit/605.1.15+(KHTML,+like+Gecko)+Version/13.1.1+Safari/605.1.15+(Applebot/0.1;++http://www.apple.com/go/applebot)",17.241.219.43,-,-,200,True,,02:30:33,Successful Investigation,301062955
113832,28/04/2024,volume=8&issue=3&article=018&target=ijor:ijaeb,"Mozilla/5.0+(Macintosh;+Intel+Mac+OS+X+10_15_5)+AppleWebKit/605.1.15+(KHTML,+like+Gecko)+Version/13.1.1+Safari/605.1.15+(Applebot/0.1;++http://www.apple.com/go/applebot)",17.241.75.175,-,-,200,True,,09:30:55,Successful Investigation,301026223
113833,28/04/2024,volume=9&issue=3&article=134&target=ijor:ijphrd,"Mozilla/5.0+(Macintosh;+Intel+Mac+OS+X+10_15_5)+AppleWebKit/605.1.15+(KHTML,+like+Gecko)+Version/13.1.1+Safari/605.1.15+(Applebot/0.1;++http://www.apple.com/go/applebot)",17.241.219.202,-,-,200,True,,00:10:04,Successful Investigation,301063114


In [193]:
df_investigations = df_investigations1[df_investigations1['is_bot'] == False]
df_investigations

,date,cs-uri-query,cs(User-Agent),c-ip,cs(Cookie),cs(Referer),sc-status,is_bot,scripted,time,Query_Category,c-ip Decimal
45,28/04/2024,article=001%27&issue=1&target=ijor%3Ajmpp1&volume=6,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/70.0.3538.102+Safari/537.36",198.54.131.58,-,-,200,False,scripted,06:53:03,Successful Investigation,3325461306
46,28/04/2024,article=001%27&issue=1&target=ijor%3Apej&volume=15,Mozilla/5.0+(compatible;+Konqueror/4.4;+Linux+2.6.32-22-generic;+X11;+en_US)+KHTML/4.4.3+(like+Gecko)+Kubuntu,198.54.131.58,-,-,200,False,scripted,01:49:39,Successful Investigation,3325461306
47,28/04/2024,article=001%27&issue=2&target=ijor%3Abpaszo&volume=37a,Opera/9.80+(Windows+NT+6.1;+U;+en)+Presto/2.7.62+Version/11.01,198.54.131.58,-,-,200,False,scripted,21:19:47,Successful Investigation,3325461306
48,28/04/2024,article=001%27&issue=3&target=ijor%3Aijaeb&volume=4,Mozilla/5.0+(X11;+FreeBSD+amd64;+rv:5.0)+Gecko/20100101+Firefox/5.0,198.54.131.58,-,-,200,False,scripted,23:10:12,Successful Investigation,3325461306
49,28/04/2024,article=001%27&issue=5&target=ijor%3Arjpt&volume=13,Mozilla/5.0+(compatible;+Konqueror/3.5;+NetBSD+4.0_RC3;+X11)+KHTML/3.5.7+(like+Gecko),198.54.131.58,-,-,200,False,scripted,22:29:56,Successful Investigation,3325461306
...,...,...,...,...,...,...,...,...,...,...,...,...
113747,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=035,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",64.224.105.15,_ga=GA1.1.757572253.1710950275;+_ga_RDCC5F3593=GS1.1.1713449912.2.1.1713449933.0.0.0;+_ga_G5TCNFJCYP=GS1.1.1713449933.2.0.1713449933.0.0.0,https://scholar.google.com/,200,False,,01:02:54,Successful Investigation,1088448783
113748,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=035,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",64.224.105.15,_ga_G5TCNFJCYP=GS1.1.1713449933.2.0.1713449933.0.0.0;+ASP.NET_SessionId=5rzr3bhb4dufb1ncvyqububg;+_gid=GA1.2.588328903.1714266178;+_gat_gtag_UA_104381405_2=1;+_ga_RDCC5F3593=GS1.1.1714266177.3.0.1714266177.0.0.0;+_ga=GA1.1.757572253.1710950275,https://www.indianjournals.com/ijor.aspx?target=ijor:zijmr&volume=9&issue=3&article=035,200,False,,01:03:17,Successful Investigation,1088448783
113749,28/04/2024,target=ijor:zijmr&volume=9&issue=4&article=022,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64;+rv:125.0)+Gecko/20100101+Firefox/125.0,5.161.57.25,-,-,200,False,,07:36:08,Successful Investigation,94451993
113751,28/04/2024,target=ijor:zijmr&volume=9&issue=4&article=038,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/117.0.0.0+Safari/537.36+Edg/117.0.2045.36",20.120.134.39,-,-,200,False,,04:41:10,Successful Investigation,343442983


In [194]:

import time
import pandas as pd 
df_clients=pd.read_excel('clients excel.xlsx')
df_clients
df_investigations['client name'] = ''

# Record the start time
start_time = time.time()

# Convert 'Start Decimal' and 'End Decimal' columns to numpy arrays for faster computation
start_decimals_df_clients = df_clients['Start Decimal'].values
end_decimals_df_clients = df_clients['End Decimal'].values
client_names_df_clients = df_clients['client name'].values

# Initialize the 'process' column to None
df_investigations['process'] = ''

# Iterate over each row in df5 to find the corresponding client name
for index, row in df_investigations.iterrows():
    c_ip_decimal = row['c-ip Decimal']
    current_client_name = row['client name']
    
    # Check if the client name is blank
    if pd.isnull(current_client_name) or current_client_name.strip() == '':
        # Use numpy array operations to check if c-ip Decimal falls within the range of start and end decimals in df8
        mask = (start_decimals_df_clients <= c_ip_decimal) & (c_ip_decimal <= end_decimals_df_clients)
        
        # If there's a match, assign the corresponding client name to df5 and mark the process
        if mask.any():
            df_investigations.at[index, 'client name'] = client_names_df_clients[mask][0]
            df_investigations.at[index, 'process'] = 'First Process'
            
# Record the end time
end_time1 = time.time()

# Calculate the total execution time
execution_time1 = end_time1 - start_time

print("Total execution time:", execution_time1, "seconds")

# Display the updated df5 with the Client Name and Process columns
ip_mapping1.rename(columns={'Client Name': 'client name'}, inplace=True)
# Assuming df9 is your IP mapping DataFrame and df_downloads is your main DataFrame

# Create a dictionary mapping IP addresses to client names
ip_client_mapping = dict(zip(ip_mapping1['c-ip'], ip_mapping1['client name']))

# Iterate over each row in df_downloads to map IP addresses to client names and update the process column
for index, row in df_investigations.iterrows():
    c_ip = row['c-ip']
    current_client_name = row['client name']
    current_process = row['process']
    
    # Check if the client name is empty or NaN
    if pd.isnull(current_client_name) or current_client_name.strip() == '':
        # Check if the IP address exists in the ip_client_mapping dictionary
        if c_ip in ip_client_mapping:
            # Assign the client name
            df_investigations.at[index, 'client name'] = ip_client_mapping[c_ip]
            # Update the process column to 'process two'
            df_investigations.at[index, 'process'] = 'Second Process'

# Display the DataFrame with the updated columns
df_investigations

C:\Users\A1\AppData\Local\Temp\ipykernel_7364\1044704354.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_investigations['client name'] = ''
C:\Users\A1\AppData\Local\Temp\ipykernel_7364\1044704354.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_investigations['process'] = ''


Total execution time: 2.8016366958618164 seconds


,date,cs-uri-query,cs(User-Agent),c-ip,cs(Cookie),cs(Referer),sc-status,is_bot,scripted,time,Query_Category,c-ip Decimal,client name,process
45,28/04/2024,article=001%27&issue=1&target=ijor%3Ajmpp1&volume=6,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/70.0.3538.102+Safari/537.36",198.54.131.58,-,-,200,False,scripted,06:53:03,Successful Investigation,3325461306,,
46,28/04/2024,article=001%27&issue=1&target=ijor%3Apej&volume=15,Mozilla/5.0+(compatible;+Konqueror/4.4;+Linux+2.6.32-22-generic;+X11;+en_US)+KHTML/4.4.3+(like+Gecko)+Kubuntu,198.54.131.58,-,-,200,False,scripted,01:49:39,Successful Investigation,3325461306,,
47,28/04/2024,article=001%27&issue=2&target=ijor%3Abpaszo&volume=37a,Opera/9.80+(Windows+NT+6.1;+U;+en)+Presto/2.7.62+Version/11.01,198.54.131.58,-,-,200,False,scripted,21:19:47,Successful Investigation,3325461306,,
48,28/04/2024,article=001%27&issue=3&target=ijor%3Aijaeb&volume=4,Mozilla/5.0+(X11;+FreeBSD+amd64;+rv:5.0)+Gecko/20100101+Firefox/5.0,198.54.131.58,-,-,200,False,scripted,23:10:12,Successful Investigation,3325461306,,
49,28/04/2024,article=001%27&issue=5&target=ijor%3Arjpt&volume=13,Mozilla/5.0+(compatible;+Konqueror/3.5;+NetBSD+4.0_RC3;+X11)+KHTML/3.5.7+(like+Gecko),198.54.131.58,-,-,200,False,scripted,22:29:56,Successful Investigation,3325461306,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113747,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=035,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",64.224.105.15,_ga=GA1.1.757572253.1710950275;+_ga_RDCC5F3593=GS1.1.1713449912.2.1.1713449933.0.0.0;+_ga_G5TCNFJCYP=GS1.1.1713449933.2.0.1713449933.0.0.0,https://scholar.google.com/,200,False,,01:02:54,Successful Investigation,1088448783,,
113748,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=035,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",64.224.105.15,_ga_G5TCNFJCYP=GS1.1.1713449933.2.0.1713449933.0.0.0;+ASP.NET_SessionId=5rzr3bhb4dufb1ncvyqububg;+_gid=GA1.2.588328903.1714266178;+_gat_gtag_UA_104381405_2=1;+_ga_RDCC5F3593=GS1.1.1714266177.3.0.1714266177.0.0.0;+_ga=GA1.1.757572253.1710950275,https://www.indianjournals.com/ijor.aspx?target=ijor:zijmr&volume=9&issue=3&article=035,200,False,,01:03:17,Successful Investigation,1088448783,,
113749,28/04/2024,target=ijor:zijmr&volume=9&issue=4&article=022,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64;+rv:125.0)+Gecko/20100101+Firefox/125.0,5.161.57.25,-,-,200,False,,07:36:08,Successful Investigation,94451993,,
113751,28/04/2024,target=ijor:zijmr&volume=9&issue=4&article=038,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/117.0.0.0+Safari/537.36+Edg/117.0.2045.36",20.120.134.39,-,-,200,False,,04:41:10,Successful Investigation,343442983,,


In [196]:
df_IP = pd.read_csv('iptolocation.csv')

# Define the mapping of old column names to new column names
column_mapping = {
    '0': 'start_ip',
    '16777215': 'end_ip',
    '-': 'country code ',
    '-.1': 'country',
    'Broadcast RFC1700':'ISP'
}

# Rename the columns using the mapping
df_IP.rename(columns=column_mapping, inplace=True)

start_time0 = time.time()

# Convert 'start_ip' and 'end_ip' columns to numpy arrays for faster computation
start_ips_df5 = df_IP['start_ip'].values
end_ips_df5 = df_IP['end_ip'].values
client_names_df5 = df_IP['ISP'].values
countries_df5 = df_IP['country'].values
df_investigations0 = df_investigations[df_investigations['process'] == '']
df_investigations1 = df_investigations0.copy()
# Iterate over each row in df5 to find the corresponding client name and country
for index, row in df_investigations1.iterrows():
    c_ip_decimal = row['c-ip Decimal']
    
    # Use numpy array operations to check if c-ip Decimal falls within the range of start_ip and end_ip
    mask = (start_ips_df5 <= c_ip_decimal) & (c_ip_decimal <= end_ips_df5)
    
    # If there's a match and the client name in df_downloads is empty or NaN, update it with the client name from df9
    if mask.any() : 
        df_investigations1.at[index, 'client name'] = client_names_df5[mask][0]
        df_investigations1.at[index, 'country'] = countries_df5[mask][0]
        # Add the process information as 'Third Process'
        df_investigations1.at[index, 'process'] = 'Third Process'

# Record the end time
end_time0 = time.time()

# Calculate the total execution time
execution_time0 = end_time0 - start_time0

print("Total execution time:", execution_time0, "seconds")

# Display the updated df5 with the Client Name, Country, and Process columns
df_investigations1

Total execution time: 1088.1663935184479 seconds


,date,cs-uri-query,cs(User-Agent),c-ip,cs(Cookie),cs(Referer),sc-status,is_bot,scripted,time,Query_Category,c-ip Decimal,client name,process,country
45,28/04/2024,article=001%27&issue=1&target=ijor%3Ajmpp1&volume=6,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/70.0.3538.102+Safari/537.36",198.54.131.58,-,-,200,False,scripted,06:53:03,Successful Investigation,3325461306,Tzulo Inc.,Third Process,United States of America
46,28/04/2024,article=001%27&issue=1&target=ijor%3Apej&volume=15,Mozilla/5.0+(compatible;+Konqueror/4.4;+Linux+2.6.32-22-generic;+X11;+en_US)+KHTML/4.4.3+(like+Gecko)+Kubuntu,198.54.131.58,-,-,200,False,scripted,01:49:39,Successful Investigation,3325461306,Tzulo Inc.,Third Process,United States of America
47,28/04/2024,article=001%27&issue=2&target=ijor%3Abpaszo&volume=37a,Opera/9.80+(Windows+NT+6.1;+U;+en)+Presto/2.7.62+Version/11.01,198.54.131.58,-,-,200,False,scripted,21:19:47,Successful Investigation,3325461306,Tzulo Inc.,Third Process,United States of America
48,28/04/2024,article=001%27&issue=3&target=ijor%3Aijaeb&volume=4,Mozilla/5.0+(X11;+FreeBSD+amd64;+rv:5.0)+Gecko/20100101+Firefox/5.0,198.54.131.58,-,-,200,False,scripted,23:10:12,Successful Investigation,3325461306,Tzulo Inc.,Third Process,United States of America
49,28/04/2024,article=001%27&issue=5&target=ijor%3Arjpt&volume=13,Mozilla/5.0+(compatible;+Konqueror/3.5;+NetBSD+4.0_RC3;+X11)+KHTML/3.5.7+(like+Gecko),198.54.131.58,-,-,200,False,scripted,22:29:56,Successful Investigation,3325461306,Tzulo Inc.,Third Process,United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113747,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=035,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",64.224.105.15,_ga=GA1.1.757572253.1710950275;+_ga_RDCC5F3593=GS1.1.1713449912.2.1.1713449933.0.0.0;+_ga_G5TCNFJCYP=GS1.1.1713449933.2.0.1713449933.0.0.0,https://scholar.google.com/,200,False,,01:02:54,Successful Investigation,1088448783,Aptum Technologies,Third Process,United States of America
113748,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=035,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",64.224.105.15,_ga_G5TCNFJCYP=GS1.1.1713449933.2.0.1713449933.0.0.0;+ASP.NET_SessionId=5rzr3bhb4dufb1ncvyqububg;+_gid=GA1.2.588328903.1714266178;+_gat_gtag_UA_104381405_2=1;+_ga_RDCC5F3593=GS1.1.1714266177.3.0.1714266177.0.0.0;+_ga=GA1.1.757572253.1710950275,https://www.indianjournals.com/ijor.aspx?target=ijor:zijmr&volume=9&issue=3&article=035,200,False,,01:03:17,Successful Investigation,1088448783,Aptum Technologies,Third Process,United States of America
113749,28/04/2024,target=ijor:zijmr&volume=9&issue=4&article=022,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64;+rv:125.0)+Gecko/20100101+Firefox/125.0,5.161.57.25,-,-,200,False,,07:36:08,Successful Investigation,94451993,Hetzner Online AG,Third Process,Germany
113751,28/04/2024,target=ijor:zijmr&volume=9&issue=4&article=038,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/117.0.0.0+Safari/537.36+Edg/117.0.2045.36",20.120.134.39,-,-,200,False,,04:41:10,Successful Investigation,343442983,Microsoft Corporation,Third Process,United States of America


In [197]:
# Drop rows from df_downloads that were identified in df_downloads1
df_investigations = df_investigations.drop(df_investigations1.index)

# Concatenate df_downloads1 with the modified df_downloads
df_investigations_final = pd.concat([df_investigations, df_investigations1], ignore_index=True)
# Optionally, you can sort the DataFrame by index
df_investigations_final.sort_index(inplace=True)
df_investigations_final['ISP'] = df_investigations_final['client name'].apply(categorize_client)
df_investigations_final

,date,cs-uri-query,cs(User-Agent),c-ip,cs(Cookie),cs(Referer),sc-status,is_bot,scripted,time,Query_Category,c-ip Decimal,client name,process,country,ISP
0,28/04/2024,target=ijor%3Aija&volume=61&issue=1&article=018&utm_source=TrendMD&utm_medium=cpc&utm_campaign=Indian_Journal_of_Agronomy_TrendMD_0,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",103.181.177.44,_ga_G5TCNFJCYP=GS1.1.1713591863.2.0.1713591863.0.0.0;+_ga_RDCC5F3593=GS1.1.1714283276.3.0.1714283276.0.0.0;+_ga=GA1.2.1397553610.1712992078;+_gid=GA1.2.875291056.1714283277,-,200,False,scripted,05:51:30,Successful Investigation,1739960620,Bihar Agricultural University,First Process,NaN,Other
1,28/04/2024,target=ijor%3alr&volume=44&issue=9&article=003,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",14.139.196.182,_ga_G5TCNFJCYP=GS1.1.1714138350.1.0.1714138350.0.0.0;+_gid=GA1.2.1682517455.1714292740;+ASP.NET_SessionId=dztbapsuqt5css4kuuztuioy;+_gat_gtag_UA_104381405_2=1;+_ga_RDCC5F3593=GS1.1.1714292739.3.1.1714292941.0.0.0;+_ga=GA1.1.1911308975.1714116440,https://www.indianjournals.com/ijor.aspx?target=ijor:lr&volume=44&issue=9&article=003,200,False,scripted,08:27:43,Successful Investigation,244040886,Indian Institute Of Technology Guwahati,First Process,NaN,Other
2,28/04/2024,target=ijor%3alr&volume=44&issue=9&article=003,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",14.139.196.182,_gid=GA1.2.1682517455.1714292740;+ASP.NET_SessionId=dztbapsuqt5css4kuuztuioy;+_ga_G5TCNFJCYP=GS1.1.1714292969.2.1.1714293256.0.0.0;+_gat_gtag_UA_104381405_2=1;+_ga_RDCC5F3593=GS1.1.1714292739.3.1.1714293282.0.0.0;+_ga=GA1.1.1911308975.1714116440,https://www.indianjournals.com/ijor.aspx?target=ijor:lr&volume=44&issue=9&article=003,200,False,scripted,08:33:12,Successful Investigation,244040886,Indian Institute Of Technology Guwahati,First Process,NaN,Other
3,28/04/2024,target=ijor%3alr&volume=44&issue=9&article=003,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",14.139.196.182,_gid=GA1.2.1682517455.1714292740;+ASP.NET_SessionId=dztbapsuqt5css4kuuztuioy;+_gat_gtag_UA_104381405_2=1;+_ga_G5TCNFJCYP=GS1.1.1714292969.2.0.1714292969.0.0.0;+_ga_RDCC5F3593=GS1.1.1714292739.3.1.1714292969.0.0.0;+_ga=GA1.1.1911308975.1714116440,https://www.indianjournals.com/ijor.aspx?target=ijor%3alr&volume=44&issue=9&article=003,200,False,scripted,08:28:04,Successful Investigation,244040886,Indian Institute Of Technology Guwahati,First Process,NaN,Other
4,28/04/2024,target=ijor:abjfm&volume=11&issue=1&article=005,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/109.0.0.0+Safari/537.36",183.87.223.194,-,https://scholar.google.com/,200,False,,06:38:49,Successful Investigation,3075989442,Indian Institute Of Management,First Process,NaN,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82565,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=035,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",64.224.105.15,_ga=GA1.1.757572253.1710950275;+_ga_RDCC5F3593=GS1.1.1713449912.2.1.1713449933.0.0.0;+_ga_G5TCNFJCYP=GS1.1.1713449933.2.0.1713449933.0.0.0,https://scholar.google.com/,200,False,,01:02:54,Successful Investigation,1088448783,Aptum Technologies,Third Process,United States of America,ISP
82566,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=035,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",64.224.105.15,_ga_G5TCNFJCYP=GS1.1.1713449933.2.0.1713449933.0.0.0;+ASP.NET_SessionId=5rzr3bhb4dufb1ncvyqububg;+_gid=GA1.2.588328903.1714266178;+_gat_gtag_UA_104381405_2=1;+_ga_RDCC5F3593=GS1.1.1714266177.3.0.1714266177.0.0.0;+_ga=GA1.1.757572253.1710950275,https://www.indianjour

In [198]:
df_investigations2 = df_investigations_final.copy()
df_investigations2[['Journal_Code', 'Volume', 'Issue']] = df_investigations2['cs-uri-query'].apply(lambda query: pd.Series(extract_journal_info(query)))
# Concatenate journal code, volume, and issue columns
df_investigations2['Concatenated'] = df_investigations2['Journal_Code'] + "_" + df_investigations2['Volume'].astype(str) + "_" + df_investigations2['Issue'].astype(str)
dfj=pd.read_excel('new product feed.xlsx')
# Concatenate journal code, volume, and issue columns
dfj['Concatenated'] = dfj['TITLE_ID'] + "_" + dfj['Sample Vol'].astype(str) + "_" + dfj['Sample Issue'].astype(str)
# Iterate over the rows of df6
for index, row in df_investigations2.iterrows():
    # Get the concatenated value
    concatenated_value = row['Concatenated']
    
    # Check if the concatenated value exists in df11
    if concatenated_value in dfj['Concatenated'].values:
        # If it exists, mark it as 'sample' in df6
        df_investigations2.at[index, 'Sample'] = 'sample'
    else:
        # If it doesn't exist, leave it blank
        df_investigations2.at[index, 'Sample'] = ''
# Drop the 'sample_vol_value' and 'sample_issue_value' columns
#df_downloads2.drop(columns=['Sample Vol', 'Sample Issue'], inplace=True)
df_investigations2

,date,cs-uri-query,cs(User-Agent),c-ip,cs(Cookie),cs(Referer),sc-status,is_bot,scripted,time,...,c-ip Decimal,client name,process,country,ISP,Journal_Code,Volume,Issue,Concatenated,Sample
0,28/04/2024,target=ijor%3Aija&volume=61&issue=1&article=018&utm_source=TrendMD&utm_medium=cpc&utm_campaign=Indian_Journal_of_Agronomy_TrendMD_0,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",103.181.177.44,_ga_G5TCNFJCYP=GS1.1.1713591863.2.0.1713591863.0.0.0;+_ga_RDCC5F3593=GS1.1.1714283276.3.0.1714283276.0.0.0;+_ga=GA1.2.1397553610.1712992078;+_gid=GA1.2.875291056.1714283277,-,200,False,scripted,05:51:30,...,1739960620,Bihar Agricultural University,First Process,NaN,Other,None,61,1,NaN,
1,28/04/2024,target=ijor%3alr&volume=44&issue=9&article=003,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",14.139.196.182,_ga_G5TCNFJCYP=GS1.1.1714138350.1.0.1714138350.0.0.0;+_gid=GA1.2.1682517455.1714292740;+ASP.NET_SessionId=dztbapsuqt5css4kuuztuioy;+_gat_gtag_UA_104381405_2=1;+_ga_RDCC5F3593=GS1.1.1714292739.3.1.1714292941.0.0.0;+_ga=GA1.1.1911308975.1714116440,https://www.indianjournals.com/ijor.aspx?target=ijor:lr&volume=44&issue=9&article=003,200,False,scripted,08:27:43,...,244040886,Indian Institute Of Technology Guwahati,First Process,NaN,Other,None,44,9,NaN,
2,28/04/2024,target=ijor%3alr&volume=44&issue=9&article=003,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",14.139.196.182,_gid=GA1.2.1682517455.1714292740;+ASP.NET_SessionId=dztbapsuqt5css4kuuztuioy;+_ga_G5TCNFJCYP=GS1.1.1714292969.2.1.1714293256.0.0.0;+_gat_gtag_UA_104381405_2=1;+_ga_RDCC5F3593=GS1.1.1714292739.3.1.1714293282.0.0.0;+_ga=GA1.1.1911308975.1714116440,https://www.indianjournals.com/ijor.aspx?target=ijor:lr&volume=44&issue=9&article=003,200,False,scripted,08:33:12,...,244040886,Indian Institute Of Technology Guwahati,First Process,NaN,Other,None,44,9,NaN,
3,28/04/2024,target=ijor%3alr&volume=44&issue=9&article=003,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",14.139.196.182,_gid=GA1.2.1682517455.1714292740;+ASP.NET_SessionId=dztbapsuqt5css4kuuztuioy;+_gat_gtag_UA_104381405_2=1;+_ga_G5TCNFJCYP=GS1.1.1714292969.2.0.1714292969.0.0.0;+_ga_RDCC5F3593=GS1.1.1714292739.3.1.1714292969.0.0.0;+_ga=GA1.1.1911308975.1714116440,https://www.indianjournals.com/ijor.aspx?target=ijor%3alr&volume=44&issue=9&article=003,200,False,scripted,08:28:04,...,244040886,Indian Institute Of Technology Guwahati,First Process,NaN,Other,None,44,9,NaN,
4,28/04/2024,target=ijor:abjfm&volume=11&issue=1&article=005,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/109.0.0.0+Safari/537.36",183.87.223.194,-,https://scholar.google.com/,200,False,,06:38:49,...,3075989442,Indian Institute Of Management,First Process,NaN,Other,abjfm,11,1,abjfm_11_1,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82565,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=035,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",64.224.105.15,_ga=GA1.1.757572253.1710950275;+_ga_RDCC5F3593=GS1.1.1713449912.2.1.1713449933.0.0.0;+_ga_G5TCNFJCYP=GS1.1.1713449933.2.0.1713449933.0.0.0,https://scholar.google.com/,200,False,,01:02:54,...,1088448783,Aptum Technologies,Third Process,United States of America,ISP,zijmr,9,3,zijmr_9_3,
82566,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=035,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",64.224.105.15,_ga_G5TCNFJCYP=GS1.1.1713449933.2.0.1713449933.0.0.0;+ASP.NET_SessionId=5rzr3bhb4dufb1ncvyqububg;+_gid=GA1.2.588328903.1714266178;+_gat_gtag_UA_104381405_2=1;+_ga_RDCC5F3593=GS1.1.1714266177.3.0.1714266177.0.0.0;+_ga=GA1.1.757572253.

In [199]:
df_investigations2['process'].value_counts()

process
Third Process    82315
First Process      255
Name: count, dtype: int64

In [200]:
df_investigations_false = df_investigations2[df_investigations2['is_bot'] == False]
df_investigations_false 

,date,cs-uri-query,cs(User-Agent),c-ip,cs(Cookie),cs(Referer),sc-status,is_bot,scripted,time,...,c-ip Decimal,client name,process,country,ISP,Journal_Code,Volume,Issue,Concatenated,Sample
0,28/04/2024,target=ijor%3Aija&volume=61&issue=1&article=018&utm_source=TrendMD&utm_medium=cpc&utm_campaign=Indian_Journal_of_Agronomy_TrendMD_0,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",103.181.177.44,_ga_G5TCNFJCYP=GS1.1.1713591863.2.0.1713591863.0.0.0;+_ga_RDCC5F3593=GS1.1.1714283276.3.0.1714283276.0.0.0;+_ga=GA1.2.1397553610.1712992078;+_gid=GA1.2.875291056.1714283277,-,200,False,scripted,05:51:30,...,1739960620,Bihar Agricultural University,First Process,NaN,Other,None,61,1,NaN,
1,28/04/2024,target=ijor%3alr&volume=44&issue=9&article=003,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",14.139.196.182,_ga_G5TCNFJCYP=GS1.1.1714138350.1.0.1714138350.0.0.0;+_gid=GA1.2.1682517455.1714292740;+ASP.NET_SessionId=dztbapsuqt5css4kuuztuioy;+_gat_gtag_UA_104381405_2=1;+_ga_RDCC5F3593=GS1.1.1714292739.3.1.1714292941.0.0.0;+_ga=GA1.1.1911308975.1714116440,https://www.indianjournals.com/ijor.aspx?target=ijor:lr&volume=44&issue=9&article=003,200,False,scripted,08:27:43,...,244040886,Indian Institute Of Technology Guwahati,First Process,NaN,Other,None,44,9,NaN,
2,28/04/2024,target=ijor%3alr&volume=44&issue=9&article=003,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",14.139.196.182,_gid=GA1.2.1682517455.1714292740;+ASP.NET_SessionId=dztbapsuqt5css4kuuztuioy;+_ga_G5TCNFJCYP=GS1.1.1714292969.2.1.1714293256.0.0.0;+_gat_gtag_UA_104381405_2=1;+_ga_RDCC5F3593=GS1.1.1714292739.3.1.1714293282.0.0.0;+_ga=GA1.1.1911308975.1714116440,https://www.indianjournals.com/ijor.aspx?target=ijor:lr&volume=44&issue=9&article=003,200,False,scripted,08:33:12,...,244040886,Indian Institute Of Technology Guwahati,First Process,NaN,Other,None,44,9,NaN,
3,28/04/2024,target=ijor%3alr&volume=44&issue=9&article=003,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",14.139.196.182,_gid=GA1.2.1682517455.1714292740;+ASP.NET_SessionId=dztbapsuqt5css4kuuztuioy;+_gat_gtag_UA_104381405_2=1;+_ga_G5TCNFJCYP=GS1.1.1714292969.2.0.1714292969.0.0.0;+_ga_RDCC5F3593=GS1.1.1714292739.3.1.1714292969.0.0.0;+_ga=GA1.1.1911308975.1714116440,https://www.indianjournals.com/ijor.aspx?target=ijor%3alr&volume=44&issue=9&article=003,200,False,scripted,08:28:04,...,244040886,Indian Institute Of Technology Guwahati,First Process,NaN,Other,None,44,9,NaN,
4,28/04/2024,target=ijor:abjfm&volume=11&issue=1&article=005,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/109.0.0.0+Safari/537.36",183.87.223.194,-,https://scholar.google.com/,200,False,,06:38:49,...,3075989442,Indian Institute Of Management,First Process,NaN,Other,abjfm,11,1,abjfm_11_1,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82565,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=035,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",64.224.105.15,_ga=GA1.1.757572253.1710950275;+_ga_RDCC5F3593=GS1.1.1713449912.2.1.1713449933.0.0.0;+_ga_G5TCNFJCYP=GS1.1.1713449933.2.0.1713449933.0.0.0,https://scholar.google.com/,200,False,,01:02:54,...,1088448783,Aptum Technologies,Third Process,United States of America,ISP,zijmr,9,3,zijmr_9_3,
82566,28/04/2024,target=ijor:zijmr&volume=9&issue=3&article=035,"Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+AppleWebKit/537.36+(KHTML,+like+Gecko)+Chrome/124.0.0.0+Safari/537.36",64.224.105.15,_ga_G5TCNFJCYP=GS1.1.1713449933.2.0.1713449933.0.0.0;+ASP.NET_SessionId=5rzr3bhb4dufb1ncvyqububg;+_gid=GA1.2.588328903.1714266178;+_gat_gtag_UA_104381405_2=1;+_ga_RDCC5F3593=GS1.1.1714266177.3.0.1714266177.0.0.0;+_ga=GA1.1.757572253.

In [201]:
# UDF ofr query category 
# if we romeve botd ( implications )
# investigations ( drilldown on types of query patterns )
# to determine what is to be kept in investigations 


In [202]:
# % amp to have a new column ( scripted )
# where the process isnt being followed ( flag as scripted )
# check this -- ccode=tnnmcjchn&volume=7&issue=2&article=006%20&type=subscribearticle
# if the entry doesnt start with ijor - scripted 

In [203]:
# Calulation of Downloads ,Denials and Investigations
# (with and without bots)

In [204]:
# process downloads with bots
df_downloads_First_Process = df_downloads[df_downloads['process'] == 'First Process']
df_downloads_Second_Process = df_downloads[df_downloads['process'] == 'Second Process']
df_downloads_Third_Process = df_downloads1[df_downloads1['process'] == 'Third Process']
# process downloads without bots
df_downloads_no_bots = df_downloads2[df_downloads2['is_bot'] == False]
df_downloads_first_process_no_bots = df_downloads_no_bots[df_downloads_no_bots['process'] == 'First Process']
df_downloads_second_process_no_bots = df_downloads_no_bots[df_downloads_no_bots['process'] == 'Second Process']
df_downloads_third_process_no_bots = df_downloads_no_bots[df_downloads_no_bots['process'] == 'Third Process']
# process denials with bots
df_denials_Third_Process = df_denials1[df_denials1['process'] == 'Third Process']
df_denials_Second_Process = df_denials2[df_denials2['process'] == 'Second Process']
df_denials_First_Process = df_denials_final[df_denials_final['process'] == 'First Process']
# process denials without bots
df_denials_no_bots = df_denials2[df_denials2['is_bot'] == False]
df_denials_first_process_no_bots = df_denials_no_bots[df_denials_no_bots['process'] == 'First Process']
df_denials_second_process_no_bots = df_denials_no_bots[df_denials_no_bots['process'] == 'Second Process']
df_denials_third_process_no_bots = df_denials_no_bots[df_denials_no_bots['process'] == 'Third Process']

# calculations of without bots from investigations 
successful_investigation_queries_without_bots = successful_investigation_queries[successful_investigation_queries['is_bot'] == False]
unsuccessful_investigation_queries_without_bots = unsuccessful_investigation_queries[unsuccessful_investigation_queries['is_bot'] == False]


# applying journal mapping
df_downloads_third_process_no_botsx=df_downloads_third_process_no_bots.copy()
df_downloads_third_process_no_botsx[['journal_code', 'journal_title', 'Access Type', 'isActive']] = df_downloads_third_process_no_botsx['cs-uri-query'].apply(lambda x: pd.Series(map_journal_info(x)))
df_denials_third_process_no_botsx=df_denials_third_process_no_bots.copy()
df_denials_third_process_no_botsx[['journal_code', 'journal_title', 'Access Type', 'isActive']] = df_denials_third_process_no_botsx['cs-uri-query'].apply(lambda x: pd.Series(map_journal_info(x)))


#access types categorization (downloads)
paid_access_downloads = df_downloads_third_process_no_botsx[df_downloads_third_process_no_botsx['Access Type'] == 'Paid']
Unknown_access_downloads = df_downloads_third_process_no_botsx[df_downloads_third_process_no_botsx['Access Type'] == 'Unknown']
Hybrid_access_downloads = df_downloads_third_process_no_botsx[df_downloads_third_process_no_botsx['Access Type'] == 'Hybrid']

#access types categorization (denials) 
paid_access_denials = df_denials_third_process_no_botsx[df_denials_third_process_no_botsx['Access Type'] == 'Paid']
Unknown_access_denials = df_denials_third_process_no_botsx[df_denials_third_process_no_botsx['Access Type'] == 'Unknown']
Hybrid_access_denials = df_denials_third_process_no_botsx[df_denials_third_process_no_botsx['Access Type'] == 'Hybrid']

#access types categorization (isActive)

active_downloads = df_downloads_third_process_no_botsx[df_downloads_third_process_no_botsx['isActive'] == 'Active']
unknown_downloads = df_downloads_third_process_no_botsx[df_downloads_third_process_no_botsx['isActive'] == 'Unknown']

#access types categorization (denials) 
active_denials = df_denials_third_process_no_botsx[df_denials_third_process_no_botsx['isActive'] == 'Active']
Unknown_denials = df_denials_third_process_no_botsx[df_denials_third_process_no_botsx['isActive'] == 'Unknown']


In [205]:
#investigations without bots  - 
#first second process , third process 



In [206]:
# CATEGORIZATION OF DOWNLOADS AND DENIALS 
# Filter the DataFrame for bots
df_bots = df_downloads_and_denials[df_downloads_and_denials['is_bot'] == True]

# Filter the DataFrame for downloads in the first and second process
df_downloads_first_second_process = df_downloads_no_bots[df_downloads_no_bots['process'] != 'Third Process']


# Filter the DataFrame for downloads and denials in the third process
df_downloads_third_process = df_downloads_no_bots[df_downloads_no_bots['process'] == 'Third Process']

# same for denials 
df_denials_first_second_process =df_denials_no_bots[df_denials_no_bots['process'] != 'Third Process']

df_denials_third_process = df_denials_no_bots[df_denials_no_bots['process'] == 'Third Process']



# only investigations without bots 
df_investigations_false = df_investigations2[df_investigations2['is_bot'] == False]
df_investigations_first_second_process = df_investigations_false[df_investigations_false['process'] != 'Third Process']
df_investigations_Third_process =  df_investigations_false[df_investigations_false['process'] == 'Third Process']                                    



In [207]:
# Rename columns ( Nomenclature )
df_bots = df_bots.rename(columns={'date': 'Date', 'cs-uri-query': 'Query', 'cs(User-Agent)': 'User_Agent',
                          'c-ip': 'IP_Address','sc-status': 'Status_Code', 'is_bot': 'BOTS', 'time': 'Time',
                          'Query_Category': 'Category', 'c-ip Decimal': 'IP_Decimal',
                          'client name': 'Client_Name', 'process': 'IP Identifier', 'country': 'Country',
                          'scripted': 'Scripted',
                          'ISP': 'ISP'})
# Rename columns ( Nomenclature ) 
df_downloads_first_second_process = df_downloads_first_second_process.rename(columns={'date': 'Date', 'cs-uri-query': 'Query', 'cs(User-Agent)': 'User_Agent',
                          'c-ip': 'IP_Address','sc-status': 'Status_Code', 'is_bot': 'BOTS', 'time': 'Time',
                          'Query_Category': 'Category', 'c-ip Decimal': 'IP_Decimal',
                          'client name': 'Client_Name', 'process': 'IP Identifier', 'country': 'Country','scripted': 'Scripted',
                          'ISP': 'ISP'})
# Rename columns ( Nomenclature )
df_downloads_third_process_no_bots = df_downloads_third_process_no_bots.rename(columns={'date': 'Date', 'cs-uri-query': 'Query', 'cs(User-Agent)': 'User_Agent',
                          'c-ip': 'IP_Address','sc-status': 'Status_Code', 'is_bot': 'BOTS', 'time': 'Time',
                          'Query_Category': 'Category', 'c-ip Decimal': 'IP_Decimal',
                          'client name': 'Client_Name', 'process': 'IP Identifier', 'country': 'Country',
                          'Sample': 'Sample', 'scripted': 'Scripted',
                          'ISP': 'ISP'})
# Rename columns ( Nomenclature )
df_denials_first_second_process = df_denials_first_second_process.rename(columns={'date': 'Date', 'cs-uri-query': 'Query', 'cs(User-Agent)': 'User_Agent',
                          'c-ip': 'IP_Address','sc-status': 'Status_Code', 'is_bot': 'BOTS', 'time': 'Time',
                          'Query_Category': 'Category', 'c-ip Decimal': 'IP_Decimal',
                          'client name': 'Client_Name', 'process': 'IP Identifier', 'country': 'Country',
                          'scripted': 'Scripted',
                          'ISP': 'ISP'})
# Rename columns ( Nomenclature )
df_denials_third_process = df_denials_third_process.rename(columns={'date': 'Date', 'cs-uri-query': 'Query', 'cs(User-Agent)': 'User_Agent',
                          'c-ip': 'IP_Address','sc-status': 'Status_Code', 'is_bot': 'BOTS', 'time': 'Time',
                          'Query_Category': 'Category', 'c-ip Decimal': 'IP_Decimal',
                          'client name': 'Client_Name', 'process': 'IP Identifier', 'country': 'Country','scripted': 'Scripted',
                          'ISP': 'ISP'})

df_investigations_first_second_process = df_investigations_first_second_process.rename(columns={'date': 'Date', 'cs-uri-query': 'Query', 'cs(User-Agent)': 'User_Agent',
                          'c-ip': 'IP_Address','sc-status': 'Status_Code', 'is_bot': 'BOTS', 'time': 'Time',
                          'Query_Category': 'Category', 'c-ip Decimal': 'IP_Decimal',
                          'client name': 'Client_Name', 'process': 'IP Identifier', 'country': 'Country','scripted': 'Scripted',
                          'ISP': 'ISP'})

df_investigations_Third_process = df_investigations_Third_process.rename(columns={'date': 'Date', 'cs-uri-query': 'Query', 'cs(User-Agent)': 'User_Agent',
                          'c-ip': 'IP_Address','sc-status': 'Status_Code', 'is_bot': 'BOTS', 'time': 'Time',
                          'Query_Category': 'Category', 'c-ip Decimal': 'IP_Decimal',
                          'client name': 'Client_Name', 'process': 'IP Identifier', 'country': 'Country','scripted': 'Scripted',
                          'ISP': 'ISP'})
# use this column selctor final 
df_bots = df_bots.loc[:, ['Date', 'Query', 'User_Agent', 'IP_Address','Status_Code', 'BOTS', 'Time', 'Category', 'IP_Decimal',
       'Client_Name', 'IP Identifier', 'Country', 'Journal_Code', 'Sample',
       'Scripted', 'ISP']]
df_downloads_first_second_process = df_downloads_first_second_process.loc[:, ['Date', 'Query', 'User_Agent', 'IP_Address','Status_Code', 'BOTS', 'Time', 'Category', 'IP_Decimal',
       'Client_Name', 'IP Identifier', 'Country', 'Journal_Code', 'Sample',
       'Scripted', 'ISP']]
df_downloads_third_process_no_bots = df_downloads_third_process_no_bots.loc[:, ['Date', 'Query', 'User_Agent', 'IP_Address','Status_Code', 'BOTS', 'Time', 'Category', 'IP_Decimal',
       'Client_Name', 'IP Identifier', 'Country', 'Journal_Code', 'Sample',
       'Scripted', 'ISP']]
df_denials_first_second_process = df_denials_first_second_process.loc[:, ['Date', 'Query', 'User_Agent', 'IP_Address','Status_Code', 'BOTS', 'Time', 'Category', 'IP_Decimal',
       'Client_Name', 'IP Identifier', 'Country', 'Journal_Code', 'Sample',
       'Scripted', 'ISP']]
df_denials_third_process = df_denials_third_process.loc[:, ['Date', 'Query', 'User_Agent', 'IP_Address','Status_Code', 'BOTS', 'Time', 'Category', 'IP_Decimal',
       'Client_Name', 'IP Identifier', 'Country', 'Journal_Code', 'Sample',
       'Scripted', 'ISP']]
df_investigations_first_second_process = df_investigations_first_second_process.loc[:, ['Date', 'Query', 'User_Agent', 'IP_Address','Status_Code', 'BOTS', 'Time', 'Category', 'IP_Decimal',
       'Client_Name', 'IP Identifier', 'Country', 'Journal_Code', 'Sample',
       'Scripted', 'ISP']]
df_investigations_Third_process = df_investigations_Third_process.loc[:, ['Date', 'Query', 'User_Agent', 'IP_Address','Status_Code', 'BOTS', 'Time', 'Category', 'IP_Decimal',
       'Client_Name', 'IP Identifier', 'Country', 'Journal_Code', 'Sample',
       'Scripted', 'ISP']]


In [210]:
import pyodbc
from sqlalchemy import create_engine
from sqlalchemy import create_engine, VARCHAR, FLOAT, INT, BOOLEAN
# Export the DataFrame to SQL Server
###################################################################
###################################################################

###################( sql data exporter )###########################   ( use this for appending )

###################################################################
###################################################################
# Define the server, database, username, and password
server = '192.168.3.5'
database = 'DB_Glomis_02may'
username = 'glomistest1'
password = 'glomistest1'

# Define the connection string
conn_str = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=SQL Server'
engine = create_engine(conn_str)

In [211]:
# Assuming df is your DataFrame
# Export the DataFrame to SQL Server
start_time3 = time.time()
df_bots.to_sql('A_bots', engine, if_exists='append', index=False)
print("bots done")
df_downloads_first_second_process.to_sql('A_Downloads_First_Second_Process1', engine, if_exists='append', index=False)
print("Downloads first second process done")
df_downloads_third_process.to_sql('A_Downloads_Third_process', engine, if_exists='append', index=False)
print("Downloads third process done")
df_denials_first_second_process.to_sql('A_Denials_First_Second_process', engine, if_exists='append', index=False)
print("Denials first second process done")
df_denials_third_process.to_sql('A_Denials_Third_Process ', engine, if_exists='append', index=False)
print("Denials third process done")
df_investigations_first_second_process.to_sql('A_Investigations_First_Second_Process', engine, if_exists='append', index=False)
print("Investigations first second process done")
df_investigations_Third_process.to_sql('A_Investigations_Third_process', engine, if_exists='append', index=False)
print("Investigations third process done")

bots done
Downloads first second process done
Downloads third process done
Denials first second process done
Denials third process done
Investigations first second process done
Investigations third process done


In [ ]:
end_time3 = time.time()

# Calculate the total execution time
execution_time3 = end_time3 - start_time3

print("Total execution time:", execution_time, "seconds")
